In [57]:
%pylab inline
import scipy as sp
import scipy.stats
import pandas as pd
from sklearn import model_selection, metrics, pipeline, preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

train = pd.read_csv('../input/training.txt', header=None, names=['id', 'type'])
test = pd.read_csv('../input/testing.txt', header=None, names=['id'])
ship_classes = ['trawler', 'longliner', 'seiner', 'other', 'support']
type_to_idx = {t: i for i, t in enumerate(ship_classes)}
train.type = train.type.transform(lambda x: type_to_idx[x])

kfold = model_selection.StratifiedKFold(4, shuffle=True, random_state=123)
folds = []
for train_idxs, test_idxs in kfold.split(train, train.type):
    folds.append((train.iloc[train_idxs], train.iloc[test_idxs]))

def read_track(idx):
    return pd.read_csv('../input/VesselTracks/{}.csv'.format(idx))

def save_result(ids, probas, filename):
    import csv
    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        for row_num in range(ids.shape[0]):
            for cls_num in range(5):
                writer.writerow((ids[row_num], ship_classes[cls_num], probas[row_num, cls_num]))
    
def multiclass_auc(y_true, probas):
    score = 0
    for idx, (name, weight) in enumerate([
        ('trawler', 0.4),
        ('longliner', 0.2),
        ('seiner', 0.3),
        ('other', 0),
        ('support', 0.1)
    ]):
        if weight <= 0:
            continue
        score += weight * metrics.roc_auc_score((y_true == idx).astype(int), probas[:, idx])
    return score
    
def xgb_multiclass_evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'auc', multiclass_auc(labels, preds)

def xgb_binary_evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'auc', metrics.roc_auc_score(labels, preds[:, 1])

train.head(5)

Populating the interactive namespace from numpy and matplotlib


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


id  type
0  100047     0
1  100070     1
2  100080     2
3  100093     3
4  100144     0

In [4]:
%%time

percentiles = [0, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99, 100]

def extract_percentiles_row(track):
    track.columns = ['TrackNumber', 'Time(seconds)', 'Latitude', 'Longitude', 'SOG',
       'Oceanic Depth', 'Chlorophyll Concentration', 'Salinity',
       'Water Surface Elevation', 'Sea Temperature', 'Thermocline Depth',
       'Eastward Water Velocity', 'Northward Water Velocity']
    return np.hstack([np.percentile(track[c], percentiles)] for c in [
        'SOG',
        'Oceanic Depth',
        'Chlorophyll Concentration',
        'Salinity',
        'Water Surface Elevation',
        'Sea Temperature',
        'Thermocline Depth',
        'Eastward Water Velocity',
        'Northward Water Velocity',
    ])

def extract_percentiles(idxs):
    return np.vstack([extract_percentiles_row(read_track(idx)) for idx in idxs])

train_all_percentiles, test_all_percentiles = extract_percentiles(train.id), extract_percentiles(test.id)

CPU times: user 14.9 s, sys: 1.23 s, total: 16.2 s
Wall time: 16.9 s


In [5]:
def load_fold(name):
    return np.load('../input/{}_train.npy'.format(name)), np.load('../input/{}_test.npy'.format(name))

fold_distances = [load_fold('fold_{}_distances'.format(i)) for i in range(4)]
all_fold_distances = np.vstack([f[1] for f in fold_distances])
all_fold_types = np.hstack([f[1].type.values for f in folds])

In [6]:
train_speed_matrices, test_speed_matrices = np.load('../input/train_speed_matrices.npy'), np.load('../input/test_speed_matrices.npy') 
train_speed_matrices = train_speed_matrices / np.sum(train_speed_matrices, axis=1)[:, None]
test_speed_matrices = test_speed_matrices / np.sum(test_speed_matrices, axis=1)[:, None]

In [9]:
train_speed_percentiles, test_speed_percentiles = np.load('../input/train_speed_percentiles.npy'), np.load('../input/test_speed_percentiles.npy') 
extended_features = np.hstack([train_speed_matrices, train_speed_percentiles])
extended_features = np.hstack([
    all_fold_distances,
    np.vstack([extended_features[test_idxs] for _, test_idxs in kfold.split(train, train.type)]),
])

extended_features_with_percentiles = np.hstack([
    extended_features,
    np.vstack([train_all_percentiles[test_idxs] for _, test_idxs in kfold.split(train, train.type)]),
])

def rank_average(predictions):
    return np.vstack(
        [np.mean([sp.stats.rankdata(p[:, i]) for p in predictions], axis=0)
          for i in range(5)]
    ).T

In [303]:
clf = LogisticRegression(C=1.0)
search = model_selection.GridSearchCV(
    clf,
    {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100, 200, 500, 700, 1000],
    },
    scoring='roc_auc',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(train_speed_matrices, train.type == 0)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] penalty=l1, C=0.1 ...............................................
[CV] penalty=l1, C=0.1 ...............................................
[CV] penalty=l1, C=0.1 ...............................................
[CV] penalty=l2, C=0.1 ...............................................
[CV] penalty=l2, C=0.1 ...............................................
[CV] penalty=l2, C=0.1 ...............................................
[CV] ................ penalty=l1, C=0.1, score=0.828074, total=   0.0s
[CV] penalty=l1, C=1 .................................................
[CV] penalty=l1, C=1 .................................................
[CV] ................ penalty=l2, C=0.1, score=0.879343, total=   0.0s
[CV] ................ penalty=l2, C=0.1, score=0.892622, total=   0.0s
[CV] ................ penalty=l2, C=0.1, score=0.857118, total=   0.0s
[CV] penalty=l1, C=1 .................................................
[CV] ...........

[Parallel(n_jobs=-1)]: Done  16 out of  48 | elapsed:    0.5s remaining:    1.0s


[CV] ................ penalty=l1, C=200, score=0.920512, total=   0.6s
[CV] penalty=l1, C=200 ...............................................
[CV] ................ penalty=l1, C=700, score=0.915814, total=   0.6s
[CV] penalty=l1, C=700 ...............................................
[CV] ................ penalty=l1, C=200, score=0.914282, total=   0.2s
[CV] penalty=l1, C=200 ...............................................
[CV] ................ penalty=l1, C=500, score=0.880791, total=   0.7s
[CV] penalty=l2, C=500 ...............................................
[CV] ................ penalty=l1, C=100, score=0.921692, total=   0.7s
[CV] penalty=l1, C=100 ...............................................
[CV] ................ penalty=l2, C=500, score=0.921692, total=   0.1s
[CV] ................ penalty=l1, C=500, score=0.916749, total=   0.7s
[CV] penalty=l1, C=500 ...............................................
[CV] penalty=l2, C=500 ...............................................
[CV] .

[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    2.4s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10, 100, 200, 500, 700, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=3)

In [305]:
search.best_score_

0.91290342652186185

In [431]:
clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('lr', LogisticRegression(C=1.0)),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'lr__penalty': ['l1', 'l2'],
        'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000],
    },
    scoring='roc_auc',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(train_speed_matrices, train.type == 0)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ........ lr__C=0.1, lr__penalty=l1, score=0.917118, total=   0.1s
[CV] ........ lr__C=0.1, lr__penalty=l2, score=0.918804, total=   0.1s
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ........ lr__C=0.1, lr__penalty=l1, score=0.923425, total=   0.1s
[CV] lr__C=1, lr__penalty=l2 .........................................
[CV] ........ lr

[Parallel(n_jobs=-1)]: Done  16 out of  48 | elapsed:    1.7s remaining:    3.5s


[CV] ........ lr__C=700, lr__penalty=l2, score=0.852274, total=   0.3s
[CV] lr__C=1000, lr__penalty=l1 ......................................
[CV] ........ lr__C=100, lr__penalty=l1, score=0.905558, total=   4.5s
[CV] lr__C=100, lr__penalty=l1 .......................................
[CV] ........ lr__C=200, lr__penalty=l1, score=0.905509, total=   4.8s
[CV] lr__C=200, lr__penalty=l1 .......................................
[CV] ........ lr__C=100, lr__penalty=l1, score=0.855081, total=   6.5s
[CV] lr__C=100, lr__penalty=l2 .......................................
[CV] ........ lr__C=100, lr__penalty=l2, score=0.905657, total=   0.2s
[CV] lr__C=100, lr__penalty=l2 .......................................
[CV] ........ lr__C=700, lr__penalty=l1, score=0.906222, total=   6.4s
[CV] lr__C=700, lr__penalty=l1 .......................................
[CV] ........ lr__C=100, lr__penalty=l2, score=0.877737, total=   0.2s
[CV] lr__C=100, lr__penalty=l2 .......................................
[CV] .

[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   16.9s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000], 'lr__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=3)

In [432]:
search.best_score_

0.91307150971642626

In [433]:
clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('зщдн', preprocessing.PolynomialFeatures(2)),
    ('lr', LogisticRegression(C=1.0)),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'lr__penalty': ['l1', 'l2'],
        'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000],
    },
    scoring='roc_auc',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(train_speed_matrices, train.type == 0)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ........ lr__C=0.1, lr__penalty=l1, score=0.909933, total=   1.8s
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ........ lr__C=0.1, lr__penalty=l1, score=0.906948, total=   2.1s
[CV] ........ lr__C=0.1, lr__penalty=l1, score=0.915789, total=   2.1s
[CV] lr__C=1, lr__penalty=l2 .........................................
[CV] lr__C=1, lr

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   33.1s


[CV] .......... lr__C=1, lr__penalty=l2, score=0.857441, total=  31.2s
[CV] lr__C=200, lr__penalty=l1 .......................................
[CV] .......... lr__C=1, lr__penalty=l2, score=0.860415, total=  33.7s
[CV] lr__C=200, lr__penalty=l1 .......................................
[CV] ........ lr__C=200, lr__penalty=l1, score=0.835219, total=   2.5s
[CV] lr__C=200, lr__penalty=l1 .......................................
[CV] ........ lr__C=200, lr__penalty=l1, score=0.836595, total=   3.0s
[CV] lr__C=200, lr__penalty=l2 .......................................
[CV] ........ lr__C=200, lr__penalty=l1, score=0.877015, total=   2.7s
[CV] lr__C=200, lr__penalty=l2 .......................................
[CV] ......... lr__C=10, lr__penalty=l2, score=0.866134, total=  32.4s
[CV] lr__C=200, lr__penalty=l2 .......................................
[CV] ......... lr__C=10, lr__penalty=l2, score=0.851454, total=  33.2s
[CV] lr__C=500, lr__penalty=l1 .......................................
[CV] .

[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  2.0min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('зщдн', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000], 'lr__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=3)

In [434]:
search.best_score_

0.91089751902859184

In [436]:
clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('lr', LogisticRegression(C=1.0)),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'lr__penalty': ['l1', 'l2'],
        'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000],
    },
    scoring='neg_log_loss',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(train_speed_matrices, train.type)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.826716, total=   0.2s
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.849834, total=   0.2s
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.815184, total=   0.2s
[CV] lr__C=1, lr__penalty=l2 .........................................
[CV] lr__C=1, lr

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    6.8s


[CV] ........ lr__C=10, lr__penalty=l1, score=-1.305771, total=   7.1s
[CV] lr__C=500, lr__penalty=l1 .......................................
[CV] ........ lr__C=10, lr__penalty=l1, score=-1.290939, total=   7.9s
[CV] lr__C=500, lr__penalty=l2 .......................................
[CV] ....... lr__C=200, lr__penalty=l2, score=-1.772560, total=   2.9s
[CV] lr__C=200, lr__penalty=l2 .......................................


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


[CV] ....... lr__C=500, lr__penalty=l2, score=-1.970944, total=   1.2s
[CV] lr__C=500, lr__penalty=l2 .......................................
[CV] ....... lr__C=500, lr__penalty=l2, score=-1.869650, total=   1.4s
[CV] lr__C=700, lr__penalty=l1 .......................................
[CV] ....... lr__C=200, lr__penalty=l2, score=-1.647036, total=   2.7s
[CV] lr__C=700, lr__penalty=l1 .......................................


...

[CV] ....... lr__C=500, lr__penalty=l1, score=-2.485594, total=  52.6s


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


[CV] ....... lr__C=700, lr__penalty=l1, score=-2.556341, total= 1.0min


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  2.5min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000], 'lr__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=3)

In [437]:
search.best_score_

-0.8051080256887746

In [438]:
clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('lr', LogisticRegression(C=1.0)),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'lr__penalty': ['l1', 'l2'],
        'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000],
    },
    scoring='neg_log_loss',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(extended_features, all_fold_types)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.661924, total=   0.8s
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.687162, total=   0.9s
[CV] lr__C=1, lr__penalty=l2 .........................................
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.575965, total=   0.9s
[CV] lr__C=1, lr

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   14.4s


[CV] ....... lr__C=100, lr__penalty=l1, score=-2.479349, total=   8.1s
[CV] lr__C=200, lr__penalty=l1 .......................................
[CV] ....... lr__C=100, lr__penalty=l1, score=-2.291057, total=   8.4s
[CV] lr__C=200, lr__penalty=l1 .......................................
[CV] ........ lr__C=10, lr__penalty=l2, score=-1.770793, total=  12.5s
[CV] lr__C=200, lr__penalty=l1 .......................................
[CV] ........ lr__C=10, lr__penalty=l2, score=-1.285876, total=  13.9s
[CV] lr__C=200, lr__penalty=l2 .......................................
[CV] ........ lr__C=10, lr__penalty=l2, score=-1.553576, total=  14.0s
[CV] lr__C=200, lr__penalty=l2 .......................................
[CV] ....... lr__C=200, lr__penalty=l1, score=-3.131819, total=   5.8s
[CV] lr__C=200, lr__penalty=l2 .......................................
[CV] ....... lr__C=200, lr__penalty=l1, score=-2.571292, total=   8.0s
[CV] lr__C=500, lr__penalty=l1 .......................................
[CV] .

[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   59.4s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000], 'lr__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=3)

In [439]:
search.best_score_

-0.64191768809256977

In [440]:
lr_probas = model_selection.cross_val_predict(search.best_estimator_, extended_features, all_fold_types, cv=4, method='predict_proba')
multiclass_auc(all_fold_types, lr_probas)

0.94870254997181014

In [309]:
train_speed_matrices.shape, train.type.values.shape

((1209, 144), (1209,))

In [332]:
%%time

gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.8)


xgb_matrix = xgb.DMatrix(train_speed_matrices, (train.type == 0).astype(int))
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 100

# gbm.fit(train_speed_matrices, train.type,
#         eval_set=[(train_speed_matrices, train.type)],
#         eval_metric=xgb_multiclass_evalerror,
#         early_stopping_rounds=5)

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    stratified=True,
    nfold=10,
    early_stopping_rounds=5,
    verbose_eval=10,
    feval=xgb_binary_evalerror,
    seed=4,
)

[0]	train-auc:0.966867+0.00490727	test-auc:0.896677+0.02865
[10]	train-auc:0.993202+0.00107478	test-auc:0.938811+0.0203299
[20]	train-auc:0.996186+0.000447116	test-auc:0.94267+0.0181112
[30]	train-auc:0.997933+0.000313847	test-auc:0.946685+0.0161947
[40]	train-auc:0.999007+0.000217899	test-auc:0.947621+0.0157635
[50]	train-auc:0.999608+9.77566e-05	test-auc:0.949652+0.016167
[60]	train-auc:0.999856+4.27604e-05	test-auc:0.950532+0.0163654
[70]	train-auc:0.999943+2.11755e-05	test-auc:0.950562+0.0154206
CPU times: user 25.2 s, sys: 8 ms, total: 25.2 s
Wall time: 3.18 s


In [348]:
%%time

gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.6,
    colsample_bytree=0.6,
    gamma=0.65,
    learning_rate=0.,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.6)


xgb_matrix = xgb.DMatrix(train_speed_matrices, train.type)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 300

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    stratified=True,
    nfold=10,
    early_stopping_rounds=5,
    verbose_eval=10,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.943461+0.00622065	test-auc:0.862178+0.0240128
[10]	train-auc:0.992454+0.000836759	test-auc:0.934838+0.0190212
[20]	train-auc:0.995475+0.000481824	test-auc:0.939306+0.0194753
[30]	train-auc:0.997234+0.000256802	test-auc:0.942724+0.0188009
[40]	train-auc:0.998324+0.000190053	test-auc:0.944186+0.0201695
[50]	train-auc:0.999037+0.000118571	test-auc:0.946187+0.01946
[60]	train-auc:0.999444+9.10335e-05	test-auc:0.947609+0.020019
[70]	train-auc:0.999708+6.28491e-05	test-auc:0.948628+0.0193857
[80]	train-auc:0.999845+3.35201e-05	test-auc:0.94991+0.0188873
[90]	train-auc:0.999925+2.02089e-05	test-auc:0.950456+0.0188052
[100]	train-auc:0.999962+1.20764e-05	test-auc:0.951298+0.0185826
[110]	train-auc:0.999981+7.60263e-06	test-auc:0.951712+0.0186238
[120]	train-auc:0.999991+5.95735e-06	test-auc:0.952179+0.0187431
CPU times: user 58.1 s, sys: 40 ms, total: 58.1 s
Wall time: 7.29 s


In [350]:
train_speed_percentiles, test_speed_percentiles = np.load('../input/train_speed_percentiles.npy'), np.load('../input/test_speed_percentiles.npy') 

In [363]:
%%time

gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.7)


xgb_matrix = xgb.DMatrix(np.hstack([train_speed_matrices, train_speed_percentiles]), train.type)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 300

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    stratified=True,
    nfold=10,
    early_stopping_rounds=5,
    verbose_eval=10,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.962115+0.00238196	test-auc:0.871184+0.0321343
[10]	train-auc:0.994854+0.000544679	test-auc:0.930656+0.0236246
[20]	train-auc:0.997168+0.000288998	test-auc:0.936407+0.0217317
[30]	train-auc:0.998559+0.000210346	test-auc:0.940495+0.0208458
[40]	train-auc:0.999319+0.00012435	test-auc:0.942771+0.0196224
[50]	train-auc:0.999698+7.82161e-05	test-auc:0.944615+0.0192005
[60]	train-auc:0.999876+4.46006e-05	test-auc:0.946689+0.0194076
[70]	train-auc:0.999955+2.22e-05	test-auc:0.948514+0.019363
[80]	train-auc:0.999985+7.32393e-06	test-auc:0.949511+0.0194471
[90]	train-auc:0.999996+3.3e-06	test-auc:0.950447+0.0190482
[100]	train-auc:0.999999+2.05913e-06	test-auc:0.950996+0.018903
[110]	train-auc:1+3e-07	test-auc:0.951595+0.0185232
[120]	train-auc:1+0	test-auc:0.951784+0.0184733
[130]	train-auc:1+0	test-auc:0.952002+0.0184447
CPU times: user 2min 20s, sys: 76 ms, total: 2min 21s
Wall time: 17.7 s


In [373]:
extended_features = np.hstack([train_speed_matrices, train_speed_percentiles])
extended_features = np.hstack([
    all_fold_distances,
    np.vstack([extended_features[test_idxs] for _, test_idxs in kfold.split(train, train.type)]),
])

In [386]:
gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.7)


xgb_matrix = xgb.DMatrix(extended_features, all_fold_types)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.075
xgb_param['n_estimators'] = 300

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
#     early_stopping_rounds=5,
    stratified=True,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.982967+0.00204814	test-auc:0.936512+0.0143373
[10]	train-auc:0.999312+0.000126455	test-auc:0.970446+0.0130325
[20]	train-auc:0.999789+3.51477e-05	test-auc:0.974184+0.0111826
[30]	train-auc:0.999933+2.06787e-05	test-auc:0.97523+0.0110615
[40]	train-auc:0.999983+7.73563e-06	test-auc:0.976395+0.0110634
[50]	train-auc:0.999995+4.39204e-06	test-auc:0.97673+0.0107895
[60]	train-auc:0.999999+1.7e-06	test-auc:0.977175+0.0107593
[70]	train-auc:1+3e-07	test-auc:0.977375+0.0108683
[80]	train-auc:1+0	test-auc:0.977645+0.0107895
[90]	train-auc:1+0	test-auc:0.977777+0.0106497
[100]	train-auc:1+0	test-auc:0.977709+0.0107031
[110]	train-auc:1+0	test-auc:0.977868+0.0106833
[120]	train-auc:1+0	test-auc:0.977848+0.0107044
[130]	train-auc:1+0	test-auc:0.97782+0.0108879
[140]	train-auc:1+0	test-auc:0.977873+0.0108143
[150]	train-auc:1+0	test-auc:0.977957+0.0107629
[160]	train-auc:1+0	test-auc:0.977973+0.0108721
[170]	train-auc:1+0	test-auc:0.978008+0.0108219
[180]	train-auc:1+0	test-auc:0.9

test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0         0.936512      0.014337        0.982967       0.002048
1         0.954044      0.012543        0.993111       0.001301
2         0.960191      0.013042        0.996246       0.000676
3         0.962768      0.011891        0.997513       0.000507
4         0.965678      0.012792        0.998059       0.000475
5         0.967387      0.011612        0.998444       0.000366
6         0.968734      0.011411        0.998678       0.000236
7         0.968622      0.012432        0.998893       0.000202
8         0.969543      0.012436        0.999059       0.000160
9         0.969732      0.012642        0.999209       0.000127
10        0.970446      0.013033        0.999312       0.000126
11        0.971107      0.013064        0.999410       0.000114
12        0.971550      0.012263        0.999491       0.000082
13        0.971722      0.011995        0.999546       0.000078
14        0.972325      0.011602        0.999591       0.000066
15        0.972419      0.012195        0.999627       0.000071
16        0.972853      0.011986        0.999666       0.000060
17        0.973202      0.011201        0.999705       0.000052
18        0.973487      0.011055        0.999742       0.000048
19        0.973593      0.011278        0.999764       0.000043
20        0.974184      0.011183        0.999789       0.000035
21        0.974151      0.011123        0.999812       0.000029
22        0.974401      0.010829        0.999833       0.000029
23        0.974370      0.011126        0.999852       0.000028
24        0.974338      0.011025        0.999871       0.000025
25        0.974395      0.011067        0.999884       0.000023
26        0.974495      0.011160        0.999896       0.000022
27        0.974647      0.011120        0.999905       0.000024
28        0.974802      0.011317        0.999918       0.000023
29        0.974868      0.011083        0.999926       0.000022
..             ...           ...             ...            ...
270       0.978103      0.010773        1.000000       0.000000
271       0.978103      0.010773        1.000000       0.000000
272       0.978132      0.010746        1.000000       0.000000
273       0.978132      0.010746        1.000000       0.000000
274       0.978110      0.010752        1.000000       0.000000
275       0.978118      0.010759        1.000000       0.000000
276       0.978120      0.010823        1.000000       0.000000
277       0.978131      0.010824        1.000000       0.000000
278       0.978108      0.010838        1.000000       0.000000
279       0.978119      0.010825        1.000000       0.000000
280       0.978120      0.010851        1.000000       0.000000
281       0.978131      0.010824        1.000000       0.000000
282       0.978120      0.010851        1.000000       0.000000
283       0.978108      0.010838        1.000000       0.000000
284       0.978101      0.010875        1.000000       0.000000
285       0.978102      0.010828        1.000000       0.000000
286       0.978090      0.010830        1.000000       0.000000
287       0.978114      0.010841        1.000000       0.000000
288       0.978101      0.010816        1.000000       0.000000
289       0.978114      0.010816        1.000000       0.000000
290       0.978136      0.010797        1.000000       0.000000
291       0.978114      0.010824        1.000000       0.000000
292       0.978132      0.010831        1.000000       0.000000
293       0.978122      0.010844        1.000000       0.000000
294       0.978143      0.010847        1.000000       0.000000
295       0.978135      0.010854        1.000000       0.000000
296       0.978119      0.010841        1.000000       0.000000
297       0.978097      0.010838        1.000000       0.000000
298       0.978079      0.010801        1.000000       0.000000
299       0.978102      0.010813        1.000000       0.000000

[300 rows x 4 columns]

In [ ]:
977356
977513
978008
978166

In [383]:
2 * 0.978008 - 1

0.956016

In [13]:
%%time

gbm_args = dict(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.075,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=250,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    silent=True,
    subsample=0.7
)

models = [xgb.XGBClassifier(seed=123 + i, **gbm_args) for i in range(5)]

predictions = [model_selection.cross_val_predict(m, extended_features, all_fold_types, cv=4, method='predict_proba') for m in models]

CPU times: user 17min 57s, sys: 312 ms, total: 17min 57s
Wall time: 2min 14s


In [401]:
scores = [multiclass_auc(all_fold_types, p) for p in predictions]
print(scores)
print(np.mean(scores))

[0.97353838032854612, 0.97377028032989421, 0.97292739274321138, 0.97278890278807761, 0.97316567424479072]
0.973238126087


In [405]:
multiclass_auc(all_fold_types, np.mean(predictions, axis=0))

0.97376471664400066

In [406]:
multiclass_auc(all_fold_types, np.median(predictions, axis=0))

0.97349866446856737

In [406]:
multiclass_auc(all_fold_types, np.median(predictions, axis=0))

0.97349866446856737

In [419]:
ra = np.vstack(
    [np.mean([sp.stats.rankdata(p[:, i]) for p in predictions], axis=0)
      for i in range(5)]
).T
multiclass_auc(all_fold_types, ra)

0.97372969025518774

In [14]:
%%time

gbm_args = dict(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.075,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=250,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    silent=True,
    subsample=0.7
)

models = [xgb.XGBClassifier(seed=123 + i, **gbm_args) for i in range(15)]

for m in models:
    m.fit(extended_features, all_fold_types)

CPU times: user 19min 3s, sys: 476 ms, total: 19min 3s
Wall time: 2min 23s


In [423]:
%%time

train_dist, test_dist = load_fold('full_distances')
extended_features_test = np.hstack([test_dist, test_speed_matrices, test_speed_percentiles])
results = [m.predict_proba(extended_features_test) for m in models]
result = np.mean(results, axis=0)
save_result(test.id.values, result, '../input/submission4.csv') # 936714.70

CPU times: user 1.38 s, sys: 8 ms, total: 1.39 s
Wall time: 192 ms


In [425]:
(.93671470 + 1) / 2

0.96835735

In [449]:
extended_features_with_percentiles = np.hstack([
    extended_features,
    np.vstack([train_all_percentiles[test_idxs] for _, test_idxs in kfold.split(train, train.type)]),
])

In [462]:
gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.5,
    colsample_bytree=0.5,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.7)


xgb_matrix = xgb.DMatrix(extended_features_with_percentiles, all_fold_types)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.065
xgb_param['n_estimators'] = 300

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
#     early_stopping_rounds=5,
    stratified=True,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.981625+0.00261983	test-auc:0.920805+0.020735
[10]	train-auc:0.999272+0.000127689	test-auc:0.972552+0.014037
[20]	train-auc:0.999717+5.05827e-05	test-auc:0.975219+0.014586
[30]	train-auc:0.999878+3.0315e-05	test-auc:0.976807+0.0127627
[40]	train-auc:0.999952+1.65481e-05	test-auc:0.977628+0.0127361
[50]	train-auc:0.999981+9.11263e-06	test-auc:0.978211+0.0128711
[60]	train-auc:0.999993+4.77074e-06	test-auc:0.978631+0.0127002
[70]	train-auc:0.999998+2.01246e-06	test-auc:0.979099+0.0126654
[80]	train-auc:1+3e-07	test-auc:0.979401+0.0126901
[90]	train-auc:1+3e-07	test-auc:0.979534+0.0125001
[100]	train-auc:1+0	test-auc:0.97978+0.0124288
[110]	train-auc:1+0	test-auc:0.979924+0.01228
[120]	train-auc:1+0	test-auc:0.979952+0.0121668
[130]	train-auc:1+0	test-auc:0.980169+0.0122443
[140]	train-auc:1+0	test-auc:0.980205+0.0122791
[150]	train-auc:1+0	test-auc:0.980192+0.0122702
[160]	train-auc:1+0	test-auc:0.980103+0.0124655
[170]	train-auc:1+0	test-auc:0.980382+0.0123836
[180]	train

test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0         0.920805      0.020735        0.981625       0.002620
1         0.946746      0.022134        0.992953       0.001042
2         0.958084      0.019341        0.996119       0.000633
3         0.961520      0.017466        0.997442       0.000341
4         0.965915      0.016751        0.998112       0.000166
5         0.968202      0.015641        0.998477       0.000179
6         0.969022      0.014810        0.998707       0.000132
7         0.969285      0.014400        0.998923       0.000175
8         0.970446      0.014856        0.999040       0.000151
9         0.971779      0.013643        0.999191       0.000136
10        0.972552      0.014037        0.999272       0.000128
11        0.973272      0.014107        0.999337       0.000126
12        0.973414      0.014661        0.999428       0.000114
13        0.973771      0.014261        0.999483       0.000113
14        0.973991      0.014168        0.999519       0.000103
15        0.974123      0.014364        0.999575       0.000083
16        0.974255      0.014277        0.999614       0.000083
17        0.974340      0.014619        0.999639       0.000068
18        0.974600      0.014968        0.999669       0.000065
19        0.974939      0.014567        0.999695       0.000055
20        0.975219      0.014586        0.999717       0.000051
21        0.975265      0.014421        0.999740       0.000049
22        0.975589      0.014154        0.999758       0.000042
23        0.975979      0.014147        0.999780       0.000039
24        0.976098      0.014099        0.999804       0.000035
25        0.976124      0.014091        0.999817       0.000034
26        0.976259      0.014093        0.999828       0.000031
27        0.976416      0.013811        0.999845       0.000035
28        0.976501      0.013499        0.999855       0.000033
29        0.976721      0.012912        0.999866       0.000031
..             ...           ...             ...            ...
270       0.980361      0.012521        1.000000       0.000000
271       0.980372      0.012523        1.000000       0.000000
272       0.980357      0.012446        1.000000       0.000000
273       0.980368      0.012454        1.000000       0.000000
274       0.980369      0.012440        1.000000       0.000000
275       0.980392      0.012447        1.000000       0.000000
276       0.980394      0.012430        1.000000       0.000000
277       0.980390      0.012406        1.000000       0.000000
278       0.980423      0.012421        1.000000       0.000000
279       0.980410      0.012410        1.000000       0.000000
280       0.980421      0.012420        1.000000       0.000000
281       0.980445      0.012422        1.000000       0.000000
282       0.980430      0.012455        1.000000       0.000000
283       0.980423      0.012449        1.000000       0.000000
284       0.980416      0.012434        1.000000       0.000000
285       0.980429      0.012433        1.000000       0.000000
286       0.980426      0.012436        1.000000       0.000000
287       0.980437      0.012440        1.000000       0.000000
288       0.980437      0.012440        1.000000       0.000000
289       0.980437      0.012440        1.000000       0.000000
290       0.980448      0.012441        1.000000       0.000000
291       0.980482      0.012452        1.000000       0.000000
292       0.980482      0.012452        1.000000       0.000000
293       0.980462      0.012444        1.000000       0.000000
294       0.980462      0.012444        1.000000       0.000000
295       0.980429      0.012461        1.000000       0.000000
296       0.980413      0.012436        1.000000       0.000000
297       0.980425      0.012441        1.000000       0.000000
298       0.980420      0.012459        1.000000       0.000000
299       0.980407      0.012453        1.000000       0.000000

[300 rows x 4 columns]

In [469]:
gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.5,
    colsample_bytree=0.5,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=10,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.6)


xgb_matrix = xgb.DMatrix(extended_features_with_percentiles, all_fold_types)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.070
xgb_param['n_estimators'] = 300

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
#     early_stopping_rounds=5,
    stratified=True,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.975661+0.00320862	test-auc:0.924543+0.0162274
[10]	train-auc:0.998929+0.000119818	test-auc:0.970861+0.0139373
[20]	train-auc:0.999579+5.34274e-05	test-auc:0.975068+0.0128936
[30]	train-auc:0.999799+3.94441e-05	test-auc:0.97611+0.0126114
[40]	train-auc:0.999913+3.11647e-05	test-auc:0.976682+0.0124545
[50]	train-auc:0.999962+1.41792e-05	test-auc:0.977181+0.0119691
[60]	train-auc:0.999985+6.9e-06	test-auc:0.9773+0.0120173
[70]	train-auc:0.999994+5.14198e-06	test-auc:0.977384+0.0122122
[80]	train-auc:0.999997+3.88458e-06	test-auc:0.97727+0.0122103
[90]	train-auc:0.999999+2.1e-06	test-auc:0.97749+0.0120857
[100]	train-auc:1+4e-07	test-auc:0.977574+0.0123438
[110]	train-auc:1+0	test-auc:0.977616+0.0119764
[120]	train-auc:1+0	test-auc:0.977777+0.0122107
[130]	train-auc:1+0	test-auc:0.977755+0.0124641
[140]	train-auc:1+0	test-auc:0.977793+0.0124227
[150]	train-auc:1+0	test-auc:0.977844+0.0123743
[160]	train-auc:1+0	test-auc:0.977919+0.0122803
[170]	train-auc:1+0	test-auc:0.9780

test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0         0.924543      0.016227        0.975661       0.003209
1         0.942403      0.013112        0.990814       0.001277
2         0.955718      0.012423        0.994702       0.000613
3         0.960865      0.012910        0.996524       0.000425
4         0.962928      0.013336        0.997342       0.000367
5         0.965150      0.013511        0.997781       0.000265
6         0.966270      0.013455        0.998165       0.000247
7         0.969010      0.013294        0.998456       0.000261
8         0.969232      0.013523        0.998677       0.000212
9         0.970066      0.014071        0.998810       0.000132
10        0.970861      0.013937        0.998929       0.000120
11        0.971101      0.013502        0.999035       0.000102
12        0.971660      0.013484        0.999140       0.000104
13        0.972633      0.013414        0.999235       0.000095
14        0.973081      0.013004        0.999297       0.000103
15        0.973629      0.013117        0.999364       0.000091
16        0.974205      0.012642        0.999414       0.000084
17        0.974493      0.012681        0.999454       0.000063
18        0.974445      0.012965        0.999496       0.000063
19        0.974749      0.013074        0.999538       0.000053
20        0.975068      0.012894        0.999579       0.000053
21        0.974697      0.013233        0.999616       0.000052
22        0.974665      0.012991        0.999644       0.000055
23        0.975278      0.012737        0.999669       0.000057
24        0.975414      0.012747        0.999692       0.000051
25        0.975451      0.012322        0.999712       0.000055
26        0.975553      0.012603        0.999727       0.000050
27        0.975684      0.012639        0.999751       0.000048
28        0.975888      0.012639        0.999770       0.000041
29        0.975945      0.012501        0.999788       0.000041
..             ...           ...             ...            ...
270       0.978087      0.011975        1.000000       0.000000
271       0.978107      0.011929        1.000000       0.000000
272       0.978113      0.011980        1.000000       0.000000
273       0.978125      0.011984        1.000000       0.000000
274       0.978134      0.011981        1.000000       0.000000
275       0.978131      0.011958        1.000000       0.000000
276       0.978112      0.011958        1.000000       0.000000
277       0.978112      0.011958        1.000000       0.000000
278       0.978144      0.011930        1.000000       0.000000
279       0.978122      0.011884        1.000000       0.000000
280       0.978110      0.011965        1.000000       0.000000
281       0.978098      0.011978        1.000000       0.000000
282       0.978130      0.011960        1.000000       0.000000
283       0.978085      0.011974        1.000000       0.000000
284       0.978107      0.011967        1.000000       0.000000
285       0.978096      0.011971        1.000000       0.000000
286       0.978109      0.011962        1.000000       0.000000
287       0.978109      0.011931        1.000000       0.000000
288       0.978131      0.011937        1.000000       0.000000
289       0.978138      0.011919        1.000000       0.000000
290       0.978131      0.011937        1.000000       0.000000
291       0.978097      0.011919        1.000000       0.000000
292       0.978120      0.011926        1.000000       0.000000
293       0.978127      0.011959        1.000000       0.000000
294       0.978119      0.011957        1.000000       0.000000
295       0.978074      0.011869        1.000000       0.000000
296       0.978090      0.011864        1.000000       0.000000
297       0.978095      0.011901        1.000000       0.000000
298       0.978117      0.011907        1.000000       0.000000
299       0.978103      0.011993        1.000000       0.000000

[300 rows x 4 columns]

In [470]:
.978038 * 2 - 1

0.9560759999999999

In [ ]:
979071
979495
979833
980448

In [15]:
%%time

gbm_args_1 = dict(
    base_score=0.5,
    colsample_bylevel=0.5,
    colsample_bytree=0.5,
    gamma=0.65,
    learning_rate=0.065,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=200,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    silent=True,
    subsample=0.7
)

models_1 = [xgb.XGBClassifier(seed=123 + i, **gbm_args_1) for i in range(7)]

predictions_1 = [model_selection.cross_val_predict(m, extended_features_with_percentiles, all_fold_types, cv=5, method='predict_proba') for m in models_1]

CPU times: user 11min 21s, sys: 388 ms, total: 11min 21s
Wall time: 1min 25s


In [16]:
%%time

gbm_args_2 = dict(
    base_score=0.5,
    colsample_bylevel=0.5,
    colsample_bytree=0.5,
    gamma=0.65,
    learning_rate=0.07,
    max_delta_step=0,
    max_depth=10,
    min_child_weight=1,
    missing=None,
    n_estimators=200,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    silent=True,
    subsample=0.6
)

models_2 = [xgb.XGBClassifier(seed=1234 + i, **gbm_args_1) for i in range(7)]

predictions_2 = [model_selection.cross_val_predict(m, extended_features_with_percentiles, all_fold_types, cv=5, method='predict_proba') for m in models_2]

CPU times: user 11min 17s, sys: 364 ms, total: 11min 18s
Wall time: 1min 24s


In [474]:
scores_1 = [multiclass_auc(all_fold_types, p) for p in predictions_1]
scores_2 = [multiclass_auc(all_fold_types, p) for p in predictions_2]
print(scores_1)
print(scores_2)
print(np.mean(scores_1), np.mean(scores_2))

[0.9748523682668756, 0.97470852899597604, 0.97402197910391863, 0.97455007111944503, 0.97408375532220226, 0.97435076583917346, 0.97478225528968054]
[0.97413666025869605, 0.97343508144096857, 0.9741153579382944, 0.97480456129345294, 0.97468780869793825, 0.97486499337138044, 0.974478754480677]
0.974478531991 0.97436045964


In [494]:
print(multiclass_auc(all_fold_types, np.mean(predictions_1, axis=0)))
print(multiclass_auc(all_fold_types, np.mean(predictions_2, axis=0)))
print(multiclass_auc(all_fold_types, (np.mean(predictions_1, axis=0) + np.mean(predictions_2, axis=0)) / 2))
print(multiclass_auc(all_fold_types, np.mean([np.mean(predictions_1, axis=0), np.mean(predictions_2, axis=0), np.mean(predictions, axis=0)], axis=0)))

0.974946304601
0.974902291957
0.974986701383
0.97582116935


In [21]:
%%time
for m in models_1:
    m.fit(extended_features_with_percentiles, all_fold_types)
for m in models_2:
    m.fit(extended_features_with_percentiles, all_fold_types)

CPU times: user 5min 51s, sys: 612 ms, total: 5min 52s
Wall time: 44.1 s


In [488]:
extended_features_test.shape, extended_features_with_all_percentiles_test.shape

((1211, 893), (1211, 1028))

In [491]:
%%time

train_dist, test_dist = load_fold('full_distances')
extended_features_with_all_percentiles_test = np.hstack([test_dist, test_speed_matrices, test_speed_percentiles, test_all_percentiles])
results_0 = [m.predict_proba(extended_features_test) for m in models]
results_1 = [m.predict_proba(extended_features_with_all_percentiles_test) for m in models_1]
results_2 = [m.predict_proba(extended_features_with_all_percentiles_test) for m in models_2]
result = np.mean([np.mean(results_0, axis=0), np.mean(results_1, axis=0), np.mean(results_2, axis=0)], axis=0)
save_result(test.id.values, result, '../input/submission5.csv') # 940246.66

CPU times: user 2.73 s, sys: 0 ns, total: 2.73 s
Wall time: 359 ms


In [478]:
print(multiclass_auc(all_fold_types, rank_average(predictions_1)))
print(multiclass_auc(all_fold_types, rank_average(predictions_2)))
print(multiclass_auc(all_fold_types, rank_average(predictions_1 + predictions_2)))

0.97491893398
0.974819932296
0.974924401067


In [497]:
clf = pipeline.Pipeline([c
    ('scaler', preprocessing.StandardScaler()),
    ('lr', LogisticRegression(C=1.0)),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'lr__penalty': ['l1', 'l2'],
        'lr__C': [0.1, 1, 10, 100, 200, 500, 700, 1000],
    },
    scoring='neg_log_loss',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(extended_features_with_percentiles, all_fold_types)
print(search.best_score_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l1 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=0.1, lr__penalty=l2 .......................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] lr__C=1, lr__penalty=l1 .........................................
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.663814, total=   1.0s
[CV] lr__C=1, lr__penalty=l1 .........................................


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.673367, total=   1.0s
[CV] lr__C=1, lr__penalty=l2 .........................................
[CV] ....... lr__C=0.1, lr__penalty=l1, score=-0.572953, total=   1.1s
[CV] lr__C=1, lr__penalty=l2 .........................................


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


[CV] ......... lr__C=1, lr__penalty=l1, score=-0.798921, total=   2.0s
[CV] lr__C=1, lr__penalty=l2 .........................................
[CV] ......... lr__C=1, lr__penalty=l1, score=-0.901613, total=   2.1s
[CV] lr__C=10, lr__penalty=l1 ........................................
[CV] ......... lr__C=1, lr__penalty=l1, score=-0.664310, total=   1.9s
[CV] lr__C=10, lr__penalty=l1 ........................................


...

[CV] ....... lr__C=700, lr__penalty=l1, score=-1.565200, total=18.0min


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


[CV] ...... lr__C=1000, lr__penalty=l1, score=-2.553763, total=12.0min
[CV] ...... lr__C=1000, lr__penalty=l1, score=-2.460041, total=12.0min
[CV] ...... lr__C=1000, lr__penalty=l1, score=-1.496935, total=12.0min


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 32.3min finished


-0.636944663939


In [498]:
lr_probas_2 = model_selection.cross_val_predict(search.best_estimator_, extended_features_with_percentiles, all_fold_types, cv=5, method='predict_proba')
multiclass_auc(all_fold_types, lr_probas_2)

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


0.95107784900303649

In [423]:
%%time

train_dist, test_dist = load_fold('full_distances')
extended_features_test = np.hstack([test_dist, test_speed_matrices, test_speed_percentiles])
results = [m.predict_proba(extended_features_test) for m in models]
result = np.mean(results, axis=0)
save_result(test.id.values, result, '../input/submission4.csv') # 936714.70

CPU times: user 1.38 s, sys: 8 ms, total: 1.39 s
Wall time: 192 ms


In [514]:
%%time

from sklearn.neural_network import MLPClassifier

clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', MLPClassifier()),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'clf__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'clf__hidden_layer_sizes': [(50, ), (100, ), (20, 20), (50, 20), (50, 50), (20, 20, 20), (50, 20, 20), ]
    },
    scoring='neg_log_loss',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(extended_features_with_percentiles, all_fold_types)
print(search.best_score_)

Fitting 3 folds for each of 49 candidates, totalling 147 fits
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.001 .................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.001 .................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.001 .................
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.001 ................
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.001 ................
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.001 ................
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.001 ..............
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.001 ..............
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.001, score=-0.762334, total=   2.1s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.001 ..............
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.001, score=-0.705685, total=   3.3s
[CV] clf__hidden_layer_sizes=(50, 20), clf__alpha=0.001 ..............
[CV]  clf__hidden_layer_sizes=(100,), clf__alpha=0.001, 

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.2s


[CV]  clf__hidden_layer_sizes=(20, 20, 20), clf__alpha=0.001, score=-0.493423, total=   2.5s
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.01 .................
[CV]  clf__hidden_layer_sizes=(20, 20, 20), clf__alpha=0.001, score=-0.803999, total=   4.4s
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.01 .................
[CV]  clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=0.001, score=-0.845871, total=   3.7s
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.01 .................
[CV]  clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=0.001, score=-0.769662, total=   4.1s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.01 ...............
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.01, score=-0.805150, total=   2.7s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.01 ...............
[CV]  clf__hidden_layer_sizes=(100,), clf__alpha=0.01, score=-0.862942, total=   2.6s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.01 ...............
[CV]  clf__hidden_layer_sizes=(

[CV]  clf__hidden_layer_sizes=(20, 20), clf__alpha=1, score=-0.748595, total=   3.1s
[CV]  clf__hidden_layer_sizes=(100,), clf__alpha=1, score=-0.424129, total=   5.2s
[CV] clf__hidden_layer_sizes=(50, 50), clf__alpha=1 ..................
[CV] clf__hidden_layer_sizes=(50, 50), clf__alpha=1 ..................
[CV]  clf__hidden_layer_sizes=(20, 20), clf__alpha=1, score=-0.525665, total=   3.5s
[CV] clf__hidden_layer_sizes=(20, 20, 20), clf__alpha=1 ..............
[CV]  clf__hidden_layer_sizes=(50, 20), clf__alpha=1, score=-0.789716, total=   2.5s
[CV] clf__hidden_layer_sizes=(20, 20, 20), clf__alpha=1 ..............
[CV]  clf__hidden_layer_sizes=(20, 20), clf__alpha=1, score=-0.642414, total=   5.2s
[CV] clf__hidden_layer_sizes=(20, 20, 20), clf__alpha=1 ..............
[CV]  clf__hidden_layer_sizes=(50, 20), clf__alpha=1, score=-0.620875, total=   3.4s
[CV] clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=1 ..............
[CV]  clf__hidden_layer_sizes=(50, 50), clf__alpha=1, score=-0.609

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


...

[CV]  clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=1000, score=-1.379641, total=   9.7s


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=1000, score=-1.390708, total=   8.7s


[Parallel(n_jobs=-1)]: Done 147 out of 147 | elapsed:  2.9min finished


-0.581094148477
CPU times: user 4.32 s, sys: 2.89 s, total: 7.2 s
Wall time: 2min 52s


In [517]:
search.best_params_, search.best_score_, search.grid_scores_

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


({'clf__alpha': 1, 'clf__hidden_layer_sizes': (50, 50)},
 -0.58109414847686935,
 [mean: -0.65590, std: 0.11370, params: {'clf__hidden_layer_sizes': (50,), 'clf__alpha': 0.001},
  mean: -0.71157, std: 0.15321, params: {'clf__hidden_layer_sizes': (100,), 'clf__alpha': 0.001},
  mean: -0.64403, std: 0.09601, params: {'clf__hidden_layer_sizes': (20, 20), 'clf__alpha': 0.001},
  mean: -0.67568, std: 0.14177, params: {'clf__hidden_layer_sizes': (50, 20), 'clf__alpha': 0.001},
  mean: -0.71186, std: 0.09716, params: {'clf__hidden_layer_sizes': (50, 50), 'clf__alpha': 0.001},
  mean: -0.63805, std: 0.12776, params: {'clf__hidden_layer_sizes': (20, 20, 20), 'clf__alpha': 0.001},
  mean: -0.70314, std: 0.15224, params: {'clf__hidden_layer_sizes': (50, 20, 20), 'clf__alpha': 0.001},
  mean: -0.69465, std: 0.13636, params: {'clf__hidden_layer_sizes': (50,), 'clf__alpha': 0.01},
  mean: -0.74065, std: 0.12489, params: {'clf__hidden_layer_sizes': (100,), 'clf__alpha': 0.01},
  mean: -0.72740, std: 0

In [519]:
%%time

from sklearn.neural_network import MLPClassifier

clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', MLPClassifier()),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'clf__alpha': [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10],
        'clf__hidden_layer_sizes': [(50, ),
                                    (100, ),
                                    (200, ),
                                    (20, 20),
                                    (50, 20),
                                    (50, 50),
                                    (50, 100),
                                    (100, 100),
                                    (200, 100),
                                    (50, 20, 20),
                                    (50, 20, 20, 20),
                                   ]
    },
    scoring='neg_log_loss',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(extended_features_with_percentiles, all_fold_types)
print(search.best_score_)

Fitting 3 folds for each of 99 candidates, totalling 297 fits
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.0001 ................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.0001 ................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.0001 ................
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.0001 ...............
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.0001 ...............
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.0001 ...............
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.0001 ...............
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.0001 ...............
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.0001, score=-0.747499, total=   1.8s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.0001 ...............
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.0001, score=-0.522341, total=   3.7s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.0001 .............
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.0001

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.5s


[CV]  clf__hidden_layer_sizes=(50, 100), clf__alpha=0.0001, score=-0.836891, total=   4.9s
[CV] clf__hidden_layer_sizes=(200, 100), clf__alpha=0.0001 ...........
[CV]  clf__hidden_layer_sizes=(50, 20), clf__alpha=0.0001, score=-0.530415, total=   7.3s
[CV] clf__hidden_layer_sizes=(200, 100), clf__alpha=0.0001 ...........
[CV]  clf__hidden_layer_sizes=(100, 100), clf__alpha=0.0001, score=-0.821486, total=   4.1s
[CV] clf__hidden_layer_sizes=(200, 100), clf__alpha=0.0001 ...........
[CV]  clf__hidden_layer_sizes=(50, 50), clf__alpha=0.0001, score=-0.564918, total=   7.4s
[CV] clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=0.0001 .........
[CV]  clf__hidden_layer_sizes=(100, 100), clf__alpha=0.0001, score=-0.793606, total=   5.5s
[CV]  clf__hidden_layer_sizes=(50, 100), clf__alpha=0.0001, score=-0.504330, total=   5.6s
[CV] clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=0.0001 .........
[CV] clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=0.0001 .........
[CV]  clf__hidden_layer_size

[CV]  clf__hidden_layer_sizes=(50, 20, 20, 20), clf__alpha=0.001, score=-0.840760, total=   7.7s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.005 ..............
[CV]  clf__hidden_layer_sizes=(50, 20, 20, 20), clf__alpha=0.001, score=-0.564928, total=   8.9s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.005 ..............
[CV]  clf__hidden_layer_sizes=(200, 100), clf__alpha=0.001, score=-0.596948, total=  14.4s
[CV] clf__hidden_layer_sizes=(20, 20), clf__alpha=0.005 ..............
[CV]  clf__hidden_layer_sizes=(50, 20, 20), clf__alpha=0.001, score=-0.611554, total=  10.7s
[CV] clf__hidden_layer_sizes=(50, 20), clf__alpha=0.005 ..............
[CV]  clf__hidden_layer_sizes=(200,), clf__alpha=0.005, score=-0.847755, total=   3.3s
[CV] clf__hidden_layer_sizes=(50, 20), clf__alpha=0.005 ..............
[CV]  clf__hidden_layer_sizes=(20, 20), clf__alpha=0.005, score=-0.907681, total=   1.6s
[CV] clf__hidden_layer_sizes=(50, 20), clf__alpha=0.005 ..............
[CV]  clf__hidden_la

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.3min


...

[CV]  clf__hidden_layer_sizes=(20, 20), clf__alpha=10, score=-0.978751, total=   6.7s
[CV] clf__hidden_layer_sizes=(50, 50), clf__alpha=10 .................
[CV]  clf__hidden_layer_sizes=(50, 20), clf__alpha=10, score=-0.952325, total=   6.2s
[CV] clf__hidden_layer_sizes=(50, 50), clf__alpha=10 .................
[CV]  clf__hidden_layer_sizes=(200,), clf__alpha=10, score=-0.800035, total=  14.5s
[CV] clf__hidden_layer_sizes=(50, 100), clf__alpha=10 ................
[CV]  clf__hidden_layer_sizes=(200,), clf__alpha=10, score=-0.601485, total=  12.7s
[CV] clf__hidden_layer_sizes=(50, 100), clf__alpha=10 ................
[CV]  clf__hidden_layer_sizes=(20, 20), clf__alpha=10, score=-0.722999, total=   8.0s
[CV] clf__hidden_layer_sizes=(50, 100), clf__alpha=10 ................
[CV]  clf__hidden_layer_sizes=(200,), clf__alpha=10, score=-0.865282, total=  15.4s
[CV] clf__hidden_layer_sizes=(100, 100), clf__alpha=10 ...............
[CV]  clf__hidden_layer_sizes=(50, 20), clf__alpha=10, score=-0.

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(50, 20, 20, 20), clf__alpha=10, score=-1.366294, total=  17.5s


[Parallel(n_jobs=-1)]: Done 297 out of 297 | elapsed:  4.3min finished


-0.585168293019
CPU times: user 5.66 s, sys: 2.16 s, total: 7.82 s
Wall time: 4min 16s


In [520]:
search.best_params_, search.best_score_, search.grid_scores_

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


({'clf__alpha': 0.5, 'clf__hidden_layer_sizes': (50, 20, 20)},
 -0.58516829301900186,
 [mean: -0.71843, std: 0.14891, params: {'clf__hidden_layer_sizes': (50,), 'clf__alpha': 0.0001},
  mean: -0.70616, std: 0.12745, params: {'clf__hidden_layer_sizes': (100,), 'clf__alpha': 0.0001},
  mean: -0.73507, std: 0.18155, params: {'clf__hidden_layer_sizes': (200,), 'clf__alpha': 0.0001},
  mean: -0.68742, std: 0.12350, params: {'clf__hidden_layer_sizes': (20, 20), 'clf__alpha': 0.0001},
  mean: -0.65731, std: 0.11068, params: {'clf__hidden_layer_sizes': (50, 20), 'clf__alpha': 0.0001},
  mean: -0.69673, std: 0.12441, params: {'clf__hidden_layer_sizes': (50, 50), 'clf__alpha': 0.0001},
  mean: -0.74815, std: 0.17375, params: {'clf__hidden_layer_sizes': (50, 100), 'clf__alpha': 0.0001},
  mean: -0.72951, std: 0.11175, params: {'clf__hidden_layer_sizes': (100, 100), 'clf__alpha': 0.0001},
  mean: -0.72906, std: 0.09951, params: {'clf__hidden_layer_sizes': (200, 100), 'clf__alpha': 0.0001},
  mean:

In [522]:
%%time

from sklearn.neural_network import MLPClassifier

clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', MLPClassifier()),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'clf__alpha': [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2],
        'clf__hidden_layer_sizes': [(50, ),
                                    (100, ),
                                    (200, ),
                                    (500, ),
                                    (20, 20),
                                    (50, 20),
                                    (50, 50),
                                    (50, 100),
                                    (100, 100),
                                    (250, 250),
                                    (200, 100),
                                    (50, 20, 20),
                                    (50, 50, 50),
                                    (100, 100, 100),
                                    (50, 20, 20, 20),
                                    (50, 50, 50, 50),
                                    (100, 100, 100, 100),
                                   ]
    },
    scoring='neg_log_loss',
    n_jobs=-1,
    cv=5,
    verbose=3,
)
search.fit(extended_features_with_percentiles, all_fold_types)
print(search.best_score_, search.best_params_)

Fitting 5 folds for each of 153 candidates, totalling 765 fits
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.1 ..................
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.1 ..................
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.1 ..................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.1, score=-0.579888, total=  15.7s
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.1 ..................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.1, score=-1.316293, total=  16.1s
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.1 ..................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.1, score=

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(100,), clf__alpha=0.1, score=-0.684425, total=  46.6s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.1 ..................


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(100,), clf__alpha=0.1, score=-1.056200, total=  46.9s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.1 ..................


...

[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.5, score=-0.875442, total=  27.2s
[CV] clf__hidden_layer_sizes=(100,), clf__alpha=0.5 ..................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.5, score=-0.545667, total=  22.0s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.5 ..................


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.5, score=-0.696243, total=  30.2s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.5 ..................
[CV]  clf__hidden_layer_sizes=(100,), clf__alpha=0.5, score=-0.559589, total=  22.8s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.5 ..................
[CV]  clf__hidden_layer_sizes=(100, 100, 100, 100), clf__alpha=0.25, score=-0.824734, total=  51.0s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.5 ..................
[CV]  clf__hidden_layer_sizes=(100,), clf__alpha=0.5, score=-1.109172, total=  35.4s
[CV] clf__hidden_layer_sizes=(200,), clf__alpha=0.5 ..................


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/srv/hdd1/Proje

KeyboardInterrupt: 

In [524]:
%%time


clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', MLPClassifier()),
])
search = model_selection.GridSearchCV(
    clf,
    {
        'clf__alpha': [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5],
        'clf__hidden_layer_sizes': [(20, ),
                                    (50, ),
#                                     (50, 50),
#                                     (50, 100),
#                                     (250, 250),
#                                     (200, 100),
#                                     (100, 100, 100),
#                                     (100, 100, 100, 100),
                                   ]
    },
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=-1,
    cv=5,
    verbose=3,
)
search.fit(extended_features_with_percentiles, all_fold_types)
print(search.best_score_, search.best_params_)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.1, score=0.962794, total=   9.7s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.1, score=0.925923, total=  10.5s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.1 ...................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.1, score=0.92

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.25, score=0.940211, total=  13.6s
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.5 ...................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.1, score=0.960269, total=  21.5s
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.5 ...................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.25, score=0.973443, total=  14.8s
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.5 ...................


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   32.8s


[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.25, score=0.932789, total=  15.6s
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=0.5 ...................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.5, score=0.957525, total=  15.3s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.5 ...................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.5, score=0.923327, total=  15.5s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.5 ...................


...

[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.5, score=0.961304, total=  18.9s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.75 ..................


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.5, score=0.946506, total=  20.1s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.75 ..................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.5, score=0.967959, total=  18.2s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.75 ..................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.75, score=0.926889, total=  15.8s
[CV] clf__hidden_layer_sizes=(50,), clf__alpha=0.75 ..................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.75, score=0.932261, total=  11.1s
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=1 .....................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.75, score=0.963038, total=   9.4s
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=1 .....................
[CV]  clf__hidden_layer_sizes=(50,), clf__alpha=0.75, score=0.974378, total=  12.7s
[CV] clf__hidden_layer_sizes=(20,), clf__alpha=1 .....................
[CV]  clf__hidden_layer_sizes=(20,), clf__alpha=0.75, score=0.970650, to

[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  2.1min finished


0.958869251665 {'clf__hidden_layer_sizes': (50,), 'clf__alpha': 1.5}
CPU times: user 3.41 s, sys: 2.4 s, total: 5.81 s
Wall time: 2min 4s


In [526]:
%%time

clf = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', MLPClassifier()),
])
search = model_selection.GridSearchCV(
    clf,
    [
        {
            'clf__solver': ['lbfgs'],
            'clf__max_iter': [1000],
            'clf__hidden_layer_sizes': [(50, ),
                                        (100, ),
                                        (200, ),
                                        (500, ),
                                        (20, 20),
                                        (50, 20),
                                        (50, 50),
                                        (50, 100),
                                        (100, 100),
                                        (250, 250),
                                        (200, 100),
                                        (50, 20, 20),
                                        (50, 50, 50),
                                        (100, 100, 100),
                                        (50, 20, 20, 20),
                                        (50, 50, 50, 50),
                                        (100, 100, 100, 100),
                                       ]
        },
        {
            'clf__solver': ['adam', 'sgd'],
            'clf__max_iter': [1000],
            'clf__alpha': [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2],
            'clf__hidden_layer_sizes': [(50, ),
                                        (100, ),
                                        (200, ),
                                        (500, ),
                                        (20, 20),
                                        (50, 20),
                                        (50, 50),
                                        (50, 100),
                                        (100, 100),
                                        (250, 250),
                                        (200, 100),
                                        (50, 20, 20),
                                        (50, 50, 50),
                                        (100, 100, 100),
                                        (50, 20, 20, 20),
                                        (50, 50, 50, 50),
                                        (100, 100, 100, 100),
                                       ]
        }
    ],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=-1,
    cv=5,
    verbose=3,
)
search.fit(extended_features_with_percentiles, all_fold_types)
print(search.best_score_, search.best_params_)

Fitting 5 folds for each of 323 candidates, totalling 1615 fits
[CV] clf__hidden_layer_sizes=(50,), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(50,), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(50,), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(50,), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(50,), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(100,), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(100,), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(100,), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(50,), clf__solver=lbfgs, clf__max_iter=1000, score=0.914929, total=   5.3s
[CV] clf__hidden_layer_sizes=(100,), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(50,), clf__solver=lbfgs, clf__max_iter=1000, score=0.951555, total=   5.3s
[CV] clf__hidden_layer_sizes=(100,), clf__sol

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   31.8s


[CV]  clf__hidden_layer_sizes=(20, 20), clf__solver=lbfgs, clf__max_iter=1000, score=0.897351, total=   4.6s
[CV] clf__hidden_layer_sizes=(20, 20), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(20, 20), clf__solver=lbfgs, clf__max_iter=1000, score=0.940585, total=   4.9s
[CV] clf__hidden_layer_sizes=(50, 20), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(20, 20), clf__solver=lbfgs, clf__max_iter=1000, score=0.942600, total=   3.9s
[CV] clf__hidden_layer_sizes=(50, 20), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(20, 20), clf__solver=lbfgs, clf__max_iter=1000, score=0.946469, total=   4.2s
[CV] clf__hidden_layer_sizes=(50, 20), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(50, 20), clf__solver=lbfgs, clf__max_iter=1000, score=0.954373, total=   8.7s
[CV] clf__hidden_layer_sizes=(50, 20), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(50, 20), clf__solver=lbfgs, clf__m

[CV] clf__hidden_layer_sizes=(100, 100, 100), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(50, 50, 50), clf__solver=lbfgs, clf__max_iter=1000, score=0.946515, total=   7.0s
[CV]  clf__hidden_layer_sizes=(50, 50, 50), clf__solver=lbfgs, clf__max_iter=1000, score=0.932238, total=   8.4s
[CV] clf__hidden_layer_sizes=(100, 100, 100), clf__solver=lbfgs, clf__max_iter=1000 
[CV] clf__hidden_layer_sizes=(100, 100, 100), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(200, 100), clf__solver=lbfgs, clf__max_iter=1000, score=0.944138, total=  22.9s
[CV] clf__hidden_layer_sizes=(50, 20, 20, 20), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(200, 100), clf__solver=lbfgs, clf__max_iter=1000, score=0.952390, total=  23.1s
[CV] clf__hidden_layer_sizes=(50, 20, 20, 20), clf__solver=lbfgs, clf__max_iter=1000 
[CV]  clf__hidden_layer_sizes=(50, 50, 50), clf__solver=lbfgs, clf__max_iter=1000, score=0.960227, total=   9.6s
[CV] clf__hi

[CV] clf__hidden_layer_sizes=(200,), clf__solver=adam, clf__max_iter=1000, clf__alpha=0.1 
[CV]  clf__hidden_layer_sizes=(100,), clf__solver=sgd, clf__max_iter=1000, clf__alpha=0.1, score=0.933765, total= 2.1min
[CV] clf__hidden_layer_sizes=(200,), clf__solver=adam, clf__max_iter=1000, clf__alpha=0.1 
[CV]  clf__hidden_layer_sizes=(100,), clf__solver=sgd, clf__max_iter=1000, clf__alpha=0.1, score=0.935943, total= 2.2min
[CV] clf__hidden_layer_sizes=(200,), clf__solver=adam, clf__max_iter=1000, clf__alpha=0.1 
[CV]  clf__hidden_layer_sizes=(100,), clf__solver=sgd, clf__max_iter=1000, clf__alpha=0.1, score=0.964176, total= 2.2min
[CV] clf__hidden_layer_sizes=(200,), clf__solver=sgd, clf__max_iter=1000, clf__alpha=0.1 
[CV]  clf__hidden_layer_sizes=(100,), clf__solver=sgd, clf__max_iter=1000, clf__alpha=0.1, score=0.970179, total= 2.3min
[CV] clf__hidden_layer_sizes=(200,), clf__solver=sgd, clf__max_iter=1000, clf__alpha=0.1 
[CV]  clf__hidden_layer_sizes=(200,), clf__solver=adam, clf__ma

...

[CV]  clf__hidden_layer_sizes=(100, 100, 100, 100), clf__solver=sgd, clf__max_iter=1000, clf__alpha=2, score=0.967434, total= 2.4min


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV]  clf__hidden_layer_sizes=(100, 100, 100, 100), clf__solver=sgd, clf__max_iter=1000, clf__alpha=2, score=0.961776, total= 2.5min


[Parallel(n_jobs=-1)]: Done 1615 out of 1615 | elapsed: 450.9min finished


0.960318753393 {'clf__hidden_layer_sizes': (200,), 'clf__solver': 'sgd', 'clf__max_iter': 1000, 'clf__alpha': 1.75}
CPU times: user 2min 11s, sys: 1min 14s, total: 3min 26s
Wall time: 7h 31min 13s


# LightGBM

In [536]:
%%time
lgb_model = lgb.LGBMClassifier(learning_rate=0.05,
                               max_depth=7,
                               seed=10)
lgb_search = model_selection.GridSearchCV(
    lgb_model, 
    [{
        'n_estimators': [100],
        'learning_rate': [0.01, 0.05, 0.1]
    }],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=1,
    cv=5,
    verbose=3,
)
lgb_search.fit(extended_features_with_percentiles, all_fold_types)
print(lgb_search.best_score_, lgb_search.best_params_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] n_estimators=100, learning_rate=0.01 ............................
[CV]  n_estimators=100, learning_rate=0.01, score=0.972800, total= 1.7min
[CV] n_estimators=100, learning_rate=0.01 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV]  n_estimators=100, learning_rate=0.01, score=0.966785, total=   9.0s
[CV] n_estimators=100, learning_rate=0.01 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.9min remaining:    0.0s


[CV]  n_estimators=100, learning_rate=0.01, score=0.955152, total=   9.0s
[CV] n_estimators=100, learning_rate=0.01 ............................
[CV]  n_estimators=100, learning_rate=0.01, score=0.972112, total=   8.7s
[CV] n_estimators=100, learning_rate=0.01 ............................
[CV]  n_estimators=100, learning_rate=0.01, score=0.976999, total=   8.9s
[CV] n_estimators=100, learning_rate=0.05 ............................
[CV]  n_estimators=100, learning_rate=0.05, score=0.978989, total=   6.6s
[CV] n_estimators=100, learning_rate=0.05 ............................
[CV]  n_estimators=100, learning_rate=0.05, score=0.972068, total=   6.3s
[CV] n_estimators=100, learning_rate=0.05 ............................
[CV]  n_estimators=100, learning_rate=0.05, score=0.964470, total=   6.5s
[CV] n_estimators=100, learning_rate=0.05 ............................
[CV]  n_estimators=100, learning_rate=0.05, score=0.983471, total=   6.5s
[CV] n_estimators=100, learning_rate=0.05 ..............

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.2min finished


0.976137413773 {'n_estimators': 100, 'learning_rate': 0.05}
CPU times: user 24min 52s, sys: 1.13 s, total: 24min 53s
Wall time: 3min 19s


In [10]:
%%time
lgb_model = lgb.LGBMClassifier(learning_rate=0.05,
                               max_depth=7,
                               seed=10)
lgb_search = model_selection.GridSearchCV(
    lgb_model, 
    [{
        'max_depth': [5, 6, 7, 8],
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.055, 0.065, 0.075, 0.1]
    }],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=1,
    cv=5,
    verbose=3,
)
lgb_search.fit(extended_features_with_percentiles, all_fold_types)
print(lgb_search.best_score_, lgb_search.best_params_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.978870, total=   4.8s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s


[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.971010, total=   4.1s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.9s remaining:    0.0s


[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.966089, total=   4.1s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.982069, total=   4.2s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.981615, total=   4.1s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_depth=5, score=0.978875, total=   5.8s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_depth=5, score=0.969875, total=   5.6s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_depth=5, score=0.964146, total=   5.8s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_dept

[CV]  learning_rate=0.055, n_estimators=100, max_depth=6, score=0.982839, total=   5.1s
[CV] learning_rate=0.055, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.055, n_estimators=200, max_depth=6, score=0.977858, total=   6.6s
[CV] learning_rate=0.055, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.055, n_estimators=200, max_depth=6, score=0.969542, total=   6.3s
[CV] learning_rate=0.055, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.055, n_estimators=200, max_depth=6, score=0.963921, total=   6.5s
[CV] learning_rate=0.055, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.055, n_estimators=200, max_depth=6, score=0.982797, total=   7.1s
[CV] learning_rate=0.055, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.055, n_estimators=200, max_depth=6, score=0.983862, total=   6.5s
[CV] learning_rate=0.055, n_estimators=100, max_depth=7 ..............
[CV]  learning_rate=0.055, n_estimators=100, m

[CV]  learning_rate=0.065, n_estimators=200, max_depth=7, score=0.969526, total=   6.3s
[CV] learning_rate=0.065, n_estimators=200, max_depth=7 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=7, score=0.963754, total=   6.4s
[CV] learning_rate=0.065, n_estimators=200, max_depth=7 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=7, score=0.982396, total=   6.5s
[CV] learning_rate=0.065, n_estimators=200, max_depth=7 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=7, score=0.984319, total=   6.7s
[CV] learning_rate=0.065, n_estimators=100, max_depth=8 ..............
[CV]  learning_rate=0.065, n_estimators=100, max_depth=8, score=0.979680, total=   6.1s
[CV] learning_rate=0.065, n_estimators=100, max_depth=8 ..............
[CV]  learning_rate=0.065, n_estimators=100, max_depth=8, score=0.970710, total=   5.8s
[CV] learning_rate=0.065, n_estimators=100, max_depth=8 ..............
[CV]  learning_rate=0.065, n_estimators=100, m

[CV]  learning_rate=0.075, n_estimators=200, max_depth=8, score=0.982442, total=   6.3s
[CV] learning_rate=0.075, n_estimators=200, max_depth=8 ..............
[CV]  learning_rate=0.075, n_estimators=200, max_depth=8, score=0.984149, total=   6.3s
[CV] learning_rate=0.1, n_estimators=100, max_depth=5 ................
[CV]  learning_rate=0.1, n_estimators=100, max_depth=5, score=0.977988, total=   2.9s
[CV] learning_rate=0.1, n_estimators=100, max_depth=5 ................
[CV]  learning_rate=0.1, n_estimators=100, max_depth=5, score=0.970332, total=   2.8s
[CV] learning_rate=0.1, n_estimators=100, max_depth=5 ................
[CV]  learning_rate=0.1, n_estimators=100, max_depth=5, score=0.963447, total=   2.9s
[CV] learning_rate=0.1, n_estimators=100, max_depth=5 ................
[CV]  learning_rate=0.1, n_estimators=100, max_depth=5, score=0.980958, total=   2.9s
[CV] learning_rate=0.1, n_estimators=100, max_depth=5 ................
[CV]  learning_rate=0.1, n_estimators=100, max_depth=5

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 18.2min finished


0.976801111635 {'learning_rate': 0.075, 'n_estimators': 100, 'max_depth': 8}
CPU times: user 2h 25min 15s, sys: 9.22 s, total: 2h 25min 24s
Wall time: 18min 18s


In [11]:
%%time
lgb_model = lgb.LGBMClassifier(learning_rate=0.05,
                               max_depth=7,
                               seed=10)
lgb_search_2 = model_selection.GridSearchCV(
    lgb_model, 
    [{
        'max_depth': [5, 6, 7, 8],
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.065, 0.075]
    }],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=1,
    cv=5,
    verbose=3,
)
lgb_search_2.fit(extended_features, all_fold_types)
print(lgb_search_2.best_score_, lgb_search_2.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.977127, total=   3.6s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s remaining:    0.0s


[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.968783, total=   3.5s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.1s remaining:    0.0s


[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.965595, total=   3.5s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.978071, total=   3.6s
[CV] learning_rate=0.05, n_estimators=100, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=100, max_depth=5, score=0.981245, total=   3.5s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_depth=5, score=0.977182, total=   5.0s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_depth=5, score=0.968569, total=   4.7s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_depth=5, score=0.966544, total=   5.0s
[CV] learning_rate=0.05, n_estimators=200, max_depth=5 ...............
[CV]  learning_rate=0.05, n_estimators=200, max_dept

[CV]  learning_rate=0.065, n_estimators=100, max_depth=6, score=0.982713, total=   4.2s
[CV] learning_rate=0.065, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=6, score=0.977631, total=   5.0s
[CV] learning_rate=0.065, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=6, score=0.967241, total=   4.8s
[CV] learning_rate=0.065, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=6, score=0.964125, total=   5.4s
[CV] learning_rate=0.065, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=6, score=0.978911, total=   4.9s
[CV] learning_rate=0.065, n_estimators=200, max_depth=6 ..............
[CV]  learning_rate=0.065, n_estimators=200, max_depth=6, score=0.982860, total=   4.9s
[CV] learning_rate=0.065, n_estimators=100, max_depth=7 ..............
[CV]  learning_rate=0.065, n_estimators=100, m

[CV]  learning_rate=0.075, n_estimators=200, max_depth=7, score=0.966953, total=   4.8s
[CV] learning_rate=0.075, n_estimators=200, max_depth=7 ..............
[CV]  learning_rate=0.075, n_estimators=200, max_depth=7, score=0.963959, total=   4.9s
[CV] learning_rate=0.075, n_estimators=200, max_depth=7 ..............
[CV]  learning_rate=0.075, n_estimators=200, max_depth=7, score=0.979537, total=   5.0s
[CV] learning_rate=0.075, n_estimators=200, max_depth=7 ..............
[CV]  learning_rate=0.075, n_estimators=200, max_depth=7, score=0.982863, total=   5.2s
[CV] learning_rate=0.075, n_estimators=100, max_depth=8 ..............
[CV]  learning_rate=0.075, n_estimators=100, max_depth=8, score=0.977211, total=   5.1s
[CV] learning_rate=0.075, n_estimators=100, max_depth=8 ..............
[CV]  learning_rate=0.075, n_estimators=100, max_depth=8, score=0.968098, total=   4.5s
[CV] learning_rate=0.075, n_estimators=100, max_depth=8 ..............
[CV]  learning_rate=0.075, n_estimators=100, m

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  9.8min finished


0.975178315907 {'learning_rate': 0.075, 'n_estimators': 100, 'max_depth': 5}
CPU times: user 1h 17min 55s, sys: 5.05 s, total: 1h 18min 1s
Wall time: 9min 49s


In [25]:
%%time

lgm_args_1 = {
    'learning_rate': 0.075,
    'n_estimators': 100,
    'max_depth': 5,
}

lgm_models_1 = [lgb.LGBMClassifier(seed=123 + i, **lgm_args_1) for i in range(7)]

lgm_predictions_1 = [model_selection.cross_val_predict(m, extended_features, all_fold_types, cv=5, method='predict_proba') for m in lgm_models_1]

CPU times: user 13min 47s, sys: 888 ms, total: 13min 48s
Wall time: 1min 44s


In [26]:
%%time

lgm_args_2 = {
    'learning_rate': 0.075,
    'n_estimators': 100,
    'max_depth': 8,
}

lgm_models_2 = [lgb.LGBMClassifier(seed=123 + i, **lgm_args_2) for i in range(7)]

lgm_predictions_2 = [model_selection.cross_val_predict(m, extended_features_with_percentiles, all_fold_types, cv=5, method='predict_proba') for m in lgm_models_2]

CPU times: user 25min 36s, sys: 1.48 s, total: 25min 37s
Wall time: 3min 13s


In [27]:
lgm_scores_1 = [multiclass_auc(all_fold_types, p) for p in lgm_predictions_1]
lgm_scores_2 = [multiclass_auc(all_fold_types, p) for p in lgm_predictions_2]
print(lgm_scores_1)
print(lgm_scores_2)
print(np.mean(lgm_scores_1), np.mean(lgm_scores_2))

print(multiclass_auc(all_fold_types, np.mean(predictions_1, axis=0)))
print(multiclass_auc(all_fold_types, np.mean(predictions_2, axis=0)))
print(multiclass_auc(all_fold_types, (np.mean(predictions_1, axis=0) + np.mean(predictions_2, axis=0)) / 2))
print(multiclass_auc(all_fold_types, np.mean([np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions]], axis=0)))
print('With lightgbm', multiclass_auc(all_fold_types, np.mean([np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))

[0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242]
[0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295]
0.974470354909 0.975843143384
0.974946304601
0.974902291957
0.974986701383
0.97582116935
With lightgbm 0.976459992158


In [30]:
print('With lightgbm', multiclass_auc(all_fold_types, rank_average([np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]])))

With lightgbm 0.976166514415


# LGBM end

# KNN

In [33]:
%%time
knn = KNeighborsClassifier()
knn_search = model_selection.GridSearchCV(
    knn, 
    [{
        'n_neighbors': range(1, 25),
    }],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=1,
    cv=5,
    verbose=3,
)
knn_search.fit(train_speed_matrices, train.type)
print(knn_search.best_score_, knn_search.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] n_neighbors=1 ...................................................
[CV] .................... n_neighbors=1, score=0.770141, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .................... n_neighbors=1, score=0.793176, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .................... n_neighbors=1, score=0.821701, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .................... n_neighbors=1, score=0.765940, total=   0.0s
[CV] n_neighbors=1 ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] .................... n_neighbors=1, score=0.792763, total=   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .................... n_neighbors=2, score=0.831563, total=   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .................... n_neighbors=2, score=0.851567, total=   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .................... n_neighbors=2, score=0.870222, total=   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .................... n_neighbors=2, score=0.834493, total=   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .................... n_neighbors=2, score=0.840369, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] .................... n_neighbors=3, score=0.864058, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] .

[CV] ................... n_neighbors=13, score=0.910624, total=   0.0s
[CV] n_neighbors=13 ..................................................
[CV] ................... n_neighbors=13, score=0.890738, total=   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ................... n_neighbors=14, score=0.909145, total=   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ................... n_neighbors=14, score=0.906940, total=   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ................... n_neighbors=14, score=0.903193, total=   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ................... n_neighbors=14, score=0.910332, total=   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ................... n_neighbors=14, score=0.892593, total=   0.0s
[CV] n_neighbors=15 ..................................................
[CV] .

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   16.4s finished


In [35]:
%%time
knn = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', KNeighborsClassifier()),
])
knn_search = model_selection.GridSearchCV(
    knn, 
    [{
        'clf__n_neighbors': range(1, 25),
    }],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=1,
    cv=5,
    verbose=3,
)
knn_search.fit(train_speed_matrices, train.type)
print(knn_search.best_score_, knn_search.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.770195, total=   0.0s
[CV] clf__n_neighbors=1 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ............... clf__n_neighbors=1, score=0.821094, total=   0.0s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.823921, total=   0.0s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.815793, total=   0.0s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.776667, total=   0.0s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.829018, total=   0.0s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.853921, total=   0.0s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.870701, total=   0.0s
[CV] clf__n_neighbors=2 ..............................................
[CV] .

[CV] .............. clf__n_neighbors=12, score=0.893667, total=   0.0s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.889727, total=   0.0s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.888937, total=   0.0s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.918574, total=   0.0s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.912491, total=   0.0s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.893172, total=   0.0s
[CV] clf__n_neighbors=14 .............................................
[CV] .............. clf__n_neighbors=14, score=0.890555, total=   0.0s
[CV] clf__n_neighbors=14 .............................................
[CV] .

[CV] .............. clf__n_neighbors=24, score=0.915983, total=   0.0s
[CV] clf__n_neighbors=24 .............................................
[CV] .............. clf__n_neighbors=24, score=0.905612, total=   0.0s
[CV] clf__n_neighbors=24 .............................................
[CV] .............. clf__n_neighbors=24, score=0.895690, total=   0.0s
0.902279237005 {'clf__n_neighbors': 12}
CPU times: user 24.2 s, sys: 4 ms, total: 24.2 s
Wall time: 24.1 s


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   24.1s finished


In [38]:
%%time
knn = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', KNeighborsClassifier()),
])
knn_search = model_selection.GridSearchCV(
    knn, 
    [{
        'clf__n_neighbors': range(1, 25),
    }],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=1,
    cv=5,
    verbose=3,
)
knn_search.fit(all_fold_distances, all_fold_types)
print(knn_search.best_score_, knn_search.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.737208, total=   0.1s
[CV] clf__n_neighbors=1 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ............... clf__n_neighbors=1, score=0.706828, total=   0.1s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.757849, total=   0.1s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.714774, total=   0.1s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.751140, total=   0.1s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.764732, total=   0.1s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.764042, total=   0.1s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.813583, total=   0.1s
[CV] clf__n_neighbors=2 ..............................................
[CV] .

[CV] .............. clf__n_neighbors=12, score=0.881359, total=   0.2s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.809533, total=   0.2s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.821307, total=   0.2s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.840846, total=   0.2s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.850754, total=   0.2s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.878592, total=   0.2s
[CV] clf__n_neighbors=14 .............................................
[CV] .............. clf__n_neighbors=14, score=0.804029, total=   0.2s
[CV] clf__n_neighbors=14 .............................................
[CV] .

[CV] .............. clf__n_neighbors=24, score=0.847476, total=   0.2s
[CV] clf__n_neighbors=24 .............................................
[CV] .............. clf__n_neighbors=24, score=0.844071, total=   0.2s
[CV] clf__n_neighbors=24 .............................................
[CV] .............. clf__n_neighbors=24, score=0.865972, total=   0.2s
0.848495584343 {'clf__n_neighbors': 17}
CPU times: user 1min 26s, sys: 20 ms, total: 1min 26s
Wall time: 1min 26s


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.4min finished


In [39]:
%%time
knn = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', KNeighborsClassifier()),
])
knn_search = model_selection.GridSearchCV(
    knn, 
    [{
        'clf__n_neighbors': range(1, 25),
    }],
    scoring=metrics.make_scorer(multiclass_auc, needs_proba=True),
    n_jobs=1,
    cv=5,
    verbose=3,
)
knn_search.fit(extended_features_with_percentiles, all_fold_types)
print(knn_search.best_score_, knn_search.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.841117, total=   0.3s
[CV] clf__n_neighbors=1 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ............... clf__n_neighbors=1, score=0.814288, total=   0.3s
[CV] clf__n_neighbors=1 ..............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV] ............... clf__n_neighbors=1, score=0.820976, total=   0.3s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.850989, total=   0.3s
[CV] clf__n_neighbors=1 ..............................................
[CV] ............... clf__n_neighbors=1, score=0.837314, total=   0.3s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.875758, total=   0.3s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.855567, total=   0.3s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.882402, total=   0.3s
[CV] clf__n_neighbors=2 ..............................................
[CV] ............... clf__n_neighbors=2, score=0.889671, total=   0.3s
[CV] clf__n_neighbors=2 ..............................................
[CV] .

[CV] .............. clf__n_neighbors=13, score=0.906731, total=   0.3s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.906288, total=   0.3s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.918382, total=   0.3s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.928656, total=   0.3s
[CV] clf__n_neighbors=13 .............................................
[CV] .............. clf__n_neighbors=13, score=0.956030, total=   0.3s
[CV] clf__n_neighbors=14 .............................................
[CV] .............. clf__n_neighbors=14, score=0.908866, total=   0.3s
[CV] clf__n_neighbors=14 .............................................
[CV] .............. clf__n_neighbors=14, score=0.903535, total=   0.3s
[CV] clf__n_neighbors=14 .............................................
[CV] .

[CV] .............. clf__n_neighbors=24, score=0.909350, total=   0.3s
[CV] clf__n_neighbors=24 .............................................
[CV] .............. clf__n_neighbors=24, score=0.952205, total=   0.3s
0.924570696133 {'clf__n_neighbors': 11}
CPU times: user 2min 52s, sys: 36 ms, total: 2min 52s
Wall time: 2min 52s


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  2.9min finished


In [41]:
best_knn = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', KNeighborsClassifier(n_neighbors=11)),
])
knn_predictions = model_selection.cross_val_predict(best_knn, extended_features_with_percentiles, all_fold_types, cv=5, method='predict_proba')

In [44]:
lgm_scores_1 = [multiclass_auc(all_fold_types, p) for p in lgm_predictions_1]
lgm_scores_2 = [multiclass_auc(all_fold_types, p) for p in lgm_predictions_2]
print(lgm_scores_1)
print(lgm_scores_2)
print(np.mean(lgm_scores_1), np.mean(lgm_scores_2))

print(multiclass_auc(all_fold_types, np.mean(predictions_1, axis=0)))
print(multiclass_auc(all_fold_types, np.mean(predictions_2, axis=0)))
print(multiclass_auc(all_fold_types, (np.mean(predictions_1, axis=0) + np.mean(predictions_2, axis=0)) / 2))
print(multiclass_auc(all_fold_types, np.mean([np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions]], axis=0)))
print('With lightgbm', multiclass_auc(all_fold_types, np.mean([np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))
print('With lightgbm 2', multiclass_auc(all_fold_types, np.mean([np.mean(p, axis=0) for p in [predictions_1, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))
print('With lightgbm and knn', multiclass_auc(all_fold_types, np.mean([knn_predictions] + [np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))
print('With lightgbm and knn 2 ', multiclass_auc(all_fold_types, np.mean([knn_predictions] * 2  + [np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))
print('With lightgbm 2 and knn', multiclass_auc(all_fold_types, np.mean([knn_predictions] + [np.mean(p, axis=0) for p in [predictions_1, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))
print('With lightgbm 3 and knn', multiclass_auc(all_fold_types, np.mean([knn_predictions] + [np.mean(p, axis=0) for p in [predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))

[0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242, 0.97447035490934242]
[0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295, 0.97584314338436295]
0.974470354909 0.975843143384
0.974946304601
0.974902291957
0.974986701383
0.97582116935
With lightgbm 0.976459992158
With lightgbm 2 0.976542406719
With lightgbm and knn 0.976721917361
With lightgbm and knn 2  0.974969631218
With lightgbm 2 and knn 0.976349905932
With lightgbm 3 and knn 0.976372717439


# KNN end


# MLP mix

In [49]:
%%time
mlp_args = {'hidden_layer_sizes': (200,), 'solver': 'sgd', 'max_iter': 1200, 'alpha': 1.75}
mlp_models = [pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('clf', MLPClassifier(random_state=234+i, **mlp_args)),
]) for i in range(7)]
mlp_predictions = [model_selection.cross_val_predict(m, extended_features_with_percentiles, all_fold_types, cv=5, method='predict_proba') for m in mlp_models]

/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/srv/hdd1/Projects/e

CPU times: user 1h 38min 8s, sys: 1h 5min 38s, total: 2h 43min 47s
Wall time: 20min 30s


/srv/hdd1/Projects/env3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [52]:
mlp_scores = [multiclass_auc(all_fold_types, p) for p in mlp_predictions]
print(mlp_scores)
print(np.mean(mlp_scores))

print('With lightgbm', multiclass_auc(all_fold_types, np.mean([np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))
print('With lightgbm and knn', multiclass_auc(all_fold_types, np.mean([knn_predictions] + [np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2]], axis=0)))
print('With lightgbm, knn and mlps', multiclass_auc(all_fold_types, np.mean([knn_predictions] + [np.mean(p, axis=0) for p in [predictions_1, predictions_2, predictions, lgm_predictions_1, lgm_predictions_2, mlp_predictions]], axis=0)))

[0.95737506550001528, 0.95793727318042654, 0.9542116566033092, 0.95729029349509809, 0.95791770354453787, 0.95748531156829908, 0.95414440573105286]
0.956623101375
With lightgbm 0.976459992158
With lightgbm and knn 0.976721917361
With lightgbm, knn and mlps 0.977100575963


# MPL mix end

# TRAIN ALL

In [53]:
%%time

for m in models:
    m.fit(extended_features, all_fold_types)

for m in models_1:
    m.fit(extended_features_with_percentiles, all_fold_types)
    
for m in models_2:
    m.fit(extended_features_with_percentiles, all_fold_types)

for m in lgm_models_1:
    m.fit(extended_features, all_fold_types)

for m in lgm_models_2:
    m.fit(extended_features_with_percentiles, all_fold_types)

best_knn.fit(extended_features_with_percentiles, all_fold_types)
    
for m in mlp_models:
    m.fit(extended_features_with_percentiles, all_fold_types)

CPU times: user 47min 16s, sys: 7min 49s, total: 55min 6s
Wall time: 6min 54s


In [58]:
%%time

knn_models = [best_knn]

train_dist, test_dist = load_fold('full_distances')
extended_features_test = np.hstack([test_dist, test_speed_matrices, test_speed_percentiles])
extended_features_with_all_percentiles_test = np.hstack([test_dist, test_speed_matrices, test_speed_percentiles, test_all_percentiles])
parts_1 = [[m.predict_proba(extended_features_test) for m in ms] for ms in [models, lgm_models_1]]
parts_2 = [[m.predict_proba(extended_features_with_all_percentiles_test) for m in ms] for ms in [models_1, models_2, lgm_models_2, knn_models, mlp_models]]
result = np.mean([np.mean(p, axis=0) for p in parts_1 + parts_2], axis=0)
save_result(test.id.values, result, '../input/submission6.csv') # 931538.36

CPU times: user 5.62 s, sys: 556 ms, total: 6.18 s
Wall time: 2.33 s


In [69]:
(.931538 + 1) / 2

0.965769

# TRAIN ALL end

In [505]:
stacked_train = np.hstack([
  np.std(predictions, axis=0),
  np.mean(predictions, axis=0),
  np.median(predictions, axis=0),
  np.max(predictions, axis=0),
  np.min(predictions, axis=0),
            
    
  np.std(predictions_1, axis=0),
  np.mean(predictions_1, axis=0),
  np.median(predictions_1, axis=0),
  np.max(predictions_1, axis=0),
  np.min(predictions_1, axis=0),

  np.std(predictions_2, axis=0),
  np.mean(predictions_2, axis=0),
  np.median(predictions_2, axis=0),
  np.max(predictions_2, axis=0),
  np.min(predictions_2, axis=0),
])
stacked_train.shape

(1209, 75)

In [511]:
gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.5,
    colsample_bytree=0.5,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=5,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.6)


xgb_matrix = xgb.DMatrix(stacked_train, all_fold_types)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.5
xgb_param['learning_rate'] = 0.050
xgb_param['n_estimators'] = 100

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
#     early_stopping_rounds=5,
    stratified=True,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.97422+0.00161938	test-auc:0.959183+0.0235388
[10]	train-auc:0.990278+0.00118614	test-auc:0.969454+0.0165685
[20]	train-auc:0.992194+0.000852716	test-auc:0.970226+0.0147045
[30]	train-auc:0.993695+0.000611926	test-auc:0.971722+0.0142032
[40]	train-auc:0.994725+0.000602022	test-auc:0.971608+0.0147236
[50]	train-auc:0.995645+0.000443081	test-auc:0.971751+0.0151945
[60]	train-auc:0.996442+0.000302965	test-auc:0.972451+0.0151122
[70]	train-auc:0.997073+0.000267908	test-auc:0.972439+0.0147885
[80]	train-auc:0.997627+0.000233378	test-auc:0.971844+0.0149946
[90]	train-auc:0.998138+0.000209099	test-auc:0.971955+0.0149497


test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.959183      0.023539        0.974220       0.001619
1        0.962648      0.019684        0.982338       0.002132
2        0.965282      0.019958        0.985716       0.001594
3        0.967999      0.019724        0.987178       0.001091
4        0.968761      0.018854        0.987976       0.001265
5        0.969306      0.017011        0.988369       0.001208
6        0.969554      0.016538        0.989211       0.001143
7        0.969292      0.016605        0.989600       0.001337
8        0.969292      0.017215        0.989872       0.001106
9        0.969268      0.016594        0.990149       0.001226
10       0.969454      0.016568        0.990278       0.001186
11       0.969073      0.015992        0.990552       0.001036
12       0.968995      0.016383        0.990832       0.000949
13       0.968761      0.016602        0.991046       0.000898
14       0.968676      0.016545        0.991097       0.000925
15       0.969490      0.016242        0.991239       0.000919
16       0.970195      0.014507        0.991364       0.000999
17       0.970017      0.014357        0.991507       0.001015
18       0.970298      0.014284        0.991842       0.000974
19       0.970402      0.014455        0.992049       0.000868
20       0.970226      0.014704        0.992194       0.000853
21       0.970366      0.014698        0.992351       0.000800
22       0.970337      0.014889        0.992488       0.000782
23       0.970368      0.014708        0.992671       0.000739
24       0.971167      0.014530        0.992800       0.000723
25       0.970997      0.014813        0.992919       0.000731
26       0.970774      0.014961        0.993161       0.000708
27       0.971729      0.013934        0.993256       0.000685
28       0.971697      0.013952        0.993500       0.000589
29       0.971749      0.014007        0.993606       0.000580
..            ...           ...             ...            ...
70       0.972439      0.014789        0.997073       0.000268
71       0.972330      0.014937        0.997118       0.000268
72       0.972290      0.014912        0.997195       0.000251
73       0.972337      0.014844        0.997259       0.000249
74       0.972243      0.014990        0.997316       0.000249
75       0.972004      0.015149        0.997373       0.000240
76       0.971971      0.015093        0.997427       0.000239
77       0.971941      0.014950        0.997469       0.000241
78       0.971892      0.014879        0.997525       0.000228
79       0.971825      0.014993        0.997574       0.000226
80       0.971844      0.014995        0.997627       0.000233
81       0.971793      0.014968        0.997681       0.000234
82       0.971706      0.014999        0.997730       0.000226
83       0.971725      0.015003        0.997786       0.000223
84       0.971808      0.014977        0.997824       0.000224
85       0.971723      0.014966        0.997878       0.000225
86       0.971550      0.015073        0.997932       0.000226
87       0.971609      0.015036        0.997975       0.000221
88       0.971725      0.015082        0.998030       0.000221
89       0.971901      0.014969        0.998099       0.000210
90       0.971955      0.014950        0.998138       0.000209
91       0.971914      0.015053        0.998189       0.000201
92       0.971926      0.014953        0.998229       0.000199
93       0.971891      0.014923        0.998272       0.000197
94       0.972046      0.014962        0.998314       0.000193
95       0.971982      0.014952        0.998365       0.000186
96       0.971995      0.014982        0.998399       0.000183
97       0.971985      0.014962        0.998439       0.000181
98       0.972015      0.014936        0.998475       0.000175
99       0.971848      0.015139        0.998514       0.000172

[100 rows x 4 columns]

In [504]:
np.max(predictions, axis=0).shape

(1209, 5)

In [476]:
.974986701383  * 2 - 1

0.9499734027660001

In [399]:
%%time

gbm_args = dict(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.075,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=250,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    silent=True,
    subsample=0.7
)

models = [xgb.XGBClassifier(seed=123 + i, **gbm_args) for i in range(5)]

predictions = [model_selection.cross_val_predict(m, extended_features, all_fold_types, cv=4, method='predict_proba') for m in models]

In [390]:
extended_features_2 = np.hstack([
    all_fold_distances,
    np.vstack([train_speed_percentiles[test_idxs] for _, test_idxs in kfold.split(train, train.type)]),
])

In [392]:
gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.5)


xgb_matrix = xgb.DMatrix(extended_features_2, all_fold_types)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.075
xgb_param['n_estimators'] = 300

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
#     early_stopping_rounds=5,
    stratified=True,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.965653+0.00262603	test-auc:0.916461+0.0217351
[10]	train-auc:0.997753+0.000276468	test-auc:0.965387+0.016499
[20]	train-auc:0.998908+0.000119167	test-auc:0.969139+0.0155826
[30]	train-auc:0.999414+8.34614e-05	test-auc:0.971656+0.0140612
[40]	train-auc:0.99966+4.10872e-05	test-auc:0.972692+0.0135052
[50]	train-auc:0.999793+2.7917e-05	test-auc:0.972958+0.0128115
[60]	train-auc:0.999889+1.9966e-05	test-auc:0.973628+0.0125038
[70]	train-auc:0.999937+1.80025e-05	test-auc:0.973812+0.0136222
[80]	train-auc:0.999968+1.18072e-05	test-auc:0.974256+0.0135403
[90]	train-auc:0.999984+8.77496e-06	test-auc:0.974675+0.0132154
[100]	train-auc:0.999991+5.21536e-06	test-auc:0.974889+0.012933
[110]	train-auc:0.999995+4.25323e-06	test-auc:0.974658+0.0131154
[120]	train-auc:0.999998+2e-06	test-auc:0.974672+0.0129642
[130]	train-auc:0.999999+1.34164e-06	test-auc:0.974616+0.0131136
[140]	train-auc:1+4.58258e-07	test-auc:0.974676+0.0131555
[150]	train-auc:1+0	test-auc:0.974547+0.0131719
[160]	t

KeyboardInterrupt: 

In [60]:
np.vstack([f[0] for f in fold_distances])

array([[  1.48660687e-05,   2.04447442e+00,   4.28865815e+00, ...,
          1.20511051e+02,   1.27544828e+02,   1.62239119e+02],
       [  0.00000000e+00,   6.03208902e+00,   1.08926760e+01, ...,
          1.28350007e+02,   1.67707654e+02,   1.94842511e+02],
       [  2.91298738e+01,   5.13314189e+01,   5.24713688e+01, ...,
          1.19042607e+02,   1.62020894e+02,   1.82867780e+02],
       ..., 
       [  7.72992052e-02,   5.04540848e+01,   5.25210544e+01, ...,
          1.27062178e+02,   1.64256514e+02,   1.86334504e+02],
       [  6.54929767e-04,   3.10176628e+00,   5.53401325e+00, ...,
          1.18720650e+02,   1.25629221e+02,   1.58993076e+02],
       [  0.00000000e+00,   3.22618404e+01,   5.59320916e+01, ...,
          1.40097729e+02,   1.71239249e+02,   1.94842511e+02]])

In [25]:
%%time

gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.1,
    learning_rate=0.07,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softmax',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=0,
    silent=True,
    subsample=0.8)

fold_train, fold_test = load_fold('fold_0_distances')
gbm.fit(fold_train, folds[0][0].type, eval_set=[(fold_test, folds[0][1].type)], eval_metric='mlogloss', early_stopping_rounds=5)

[0]	validation_0-mlogloss:1.5133
Will train until validation_0-mlogloss hasn't improved in 5 rounds.
[1]	validation_0-mlogloss:1.43088
[2]	validation_0-mlogloss:1.35548
[3]	validation_0-mlogloss:1.29544
[4]	validation_0-mlogloss:1.24263
[5]	validation_0-mlogloss:1.19325
[6]	validation_0-mlogloss:1.14699
[7]	validation_0-mlogloss:1.10586
[8]	validation_0-mlogloss:1.07164
[9]	validation_0-mlogloss:1.03415
[10]	validation_0-mlogloss:1.00304
[11]	validation_0-mlogloss:0.973107
[12]	validation_0-mlogloss:0.94595
[13]	validation_0-mlogloss:0.922048
[14]	validation_0-mlogloss:0.898886
[15]	validation_0-mlogloss:0.879101
[16]	validation_0-mlogloss:0.859676
[17]	validation_0-mlogloss:0.840968
[18]	validation_0-mlogloss:0.823168
[19]	validation_0-mlogloss:0.808124
[20]	validation_0-mlogloss:0.794672
[21]	validation_0-mlogloss:0.779467
[22]	validation_0-mlogloss:0.766927
[23]	validation_0-mlogloss:0.753776
[24]	validation_0-mlogloss:0.742418
[25]	validation_0-mlogloss:0.731234
[26]	validation_0-m

In [161]:
%%time

gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softmax',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.8)


fold_train, fold_test = load_fold('fold_0_distances')
gbm.fit(fold_train, folds[0][0].type, eval_set=[(fold_test, folds[0][1].type)], eval_metric=xgb_multiclass_evalerror, early_stopping_rounds=5)

[0]	validation_0-auc:0.907541
Will train until validation_0-auc hasn't improved in 5 rounds.
[1]	validation_0-auc:0.915135
[2]	validation_0-auc:0.916833
[3]	validation_0-auc:0.924793
[4]	validation_0-auc:0.923836
[5]	validation_0-auc:0.926435
[6]	validation_0-auc:0.92838
[7]	validation_0-auc:0.930954
[8]	validation_0-auc:0.931025
[9]	validation_0-auc:0.931758
[10]	validation_0-auc:0.932602
[11]	validation_0-auc:0.932664
[12]	validation_0-auc:0.932813
[13]	validation_0-auc:0.933236
[14]	validation_0-auc:0.932999
[15]	validation_0-auc:0.933888
[16]	validation_0-auc:0.935713
[17]	validation_0-auc:0.936871
[18]	validation_0-auc:0.938966
[19]	validation_0-auc:0.938709
[20]	validation_0-auc:0.938139
[21]	validation_0-auc:0.939025
[22]	validation_0-auc:0.939721
[23]	validation_0-auc:0.939794
[24]	validation_0-auc:0.940086
[25]	validation_0-auc:0.939883
[26]	validation_0-auc:0.940094
[27]	validation_0-auc:0.939013
[28]	validation_0-auc:0.939126
[29]	validation_0-auc:0.938678
[30]	validation_0-

In [364]:
gbm = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='multi:softprob',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=1,
    silent=True,
    subsample=0.8)


xgb_matrix = xgb.DMatrix(all_fold_distances, all_fold_types)
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 5
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.65
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 100

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
#     early_stopping_rounds=5,
    feval=xgb_multiclass_evalerror,
    seed=4,
)

[0]	train-auc:0.981517+0.0019408	test-auc:0.913522+0.0151506
[10]	train-auc:0.998086+0.000181405	test-auc:0.948119+0.0126834
[20]	train-auc:0.999002+9.83049e-05	test-auc:0.951829+0.0124228
[30]	train-auc:0.999496+6.9312e-05	test-auc:0.9533+0.0128835
[40]	train-auc:0.999769+4.45521e-05	test-auc:0.954324+0.0134629
[50]	train-auc:0.999923+1.6849e-05	test-auc:0.955222+0.0141389
[60]	train-auc:0.999979+7.26705e-06	test-auc:0.955756+0.0145799
[70]	train-auc:0.999995+4.61411e-06	test-auc:0.955674+0.0151426
[80]	train-auc:0.999999+1.18743e-06	test-auc:0.955509+0.0152959
[90]	train-auc:1+0	test-auc:0.955592+0.0153078


test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.913522      0.015151        0.981517   1.940800e-03
1        0.927616      0.016859        0.991545   9.965726e-04
2        0.935682      0.017070        0.994456   6.024567e-04
3        0.939120      0.015492        0.995724   4.361688e-04
4        0.941579      0.015962        0.996527   3.628700e-04
5        0.944531      0.014753        0.996966   3.658828e-04
6        0.945395      0.015100        0.997279   2.749483e-04
7        0.945997      0.013783        0.997554   2.364533e-04
8        0.947252      0.013515        0.997796   2.143143e-04
9        0.947242      0.012887        0.997925   1.919840e-04
10       0.948119      0.012683        0.998086   1.814047e-04
11       0.948140      0.012691        0.998209   1.978082e-04
12       0.948410      0.012650        0.998331   1.772281e-04
13       0.949157      0.013143        0.998455   1.982230e-04
14       0.949465      0.013292        0.998536   1.799458e-04
15       0.950430      0.012715        0.998643   1.574085e-04
16       0.950622      0.012643        0.998711   1.606865e-04
17       0.950747      0.012559        0.998800   1.347512e-04
18       0.950868      0.012788        0.998873   1.166240e-04
19       0.951558      0.012425        0.998934   1.058484e-04
20       0.951829      0.012423        0.999002   9.830488e-05
21       0.951995      0.012357        0.999075   7.962544e-05
22       0.952661      0.012466        0.999123   8.274394e-05
23       0.952801      0.012443        0.999186   6.009168e-05
24       0.953032      0.012338        0.999239   6.908726e-05
25       0.953187      0.012305        0.999282   7.023931e-05
26       0.952905      0.012601        0.999329   6.441770e-05
27       0.953203      0.012524        0.999377   7.649895e-05
28       0.953030      0.012544        0.999407   7.483776e-05
29       0.953188      0.012822        0.999454   7.628689e-05
..            ...           ...             ...            ...
70       0.955674      0.015143        0.999995   4.614109e-06
71       0.955613      0.015037        0.999996   3.590265e-06
72       0.955840      0.015076        0.999997   3.200000e-06
73       0.955885      0.015070        0.999997   2.835489e-06
74       0.955699      0.015099        0.999998   2.332381e-06
75       0.955518      0.015120        0.999998   2.385372e-06
76       0.955432      0.015069        0.999998   2.291288e-06
77       0.955583      0.015131        0.999998   2.291288e-06
78       0.955708      0.015188        0.999999   1.513275e-06
79       0.955577      0.015177        0.999999   1.661325e-06
80       0.955509      0.015296        0.999999   1.187434e-06
81       0.955587      0.015233        0.999999   9.165151e-07
82       0.955655      0.015154        1.000000   9.165151e-07
83       0.955657      0.014994        1.000000   4.000000e-07
84       0.955743      0.014911        1.000000   4.000000e-07
85       0.955637      0.014919        1.000000   3.000000e-07
86       0.955597      0.015102        1.000000   3.000000e-07
87       0.955551      0.015310        1.000000   3.000000e-07
88       0.955693      0.015384        1.000000   0.000000e+00
89       0.955640      0.015391        1.000000   0.000000e+00
90       0.955592      0.015308        1.000000   0.000000e+00
91       0.955679      0.015317        1.000000   0.000000e+00
92       0.955653      0.015214        1.000000   0.000000e+00
93       0.955568      0.015137        1.000000   0.000000e+00
94       0.955730      0.015324        1.000000   0.000000e+00
95       0.955766      0.015098        1.000000   0.000000e+00
96       0.955810      0.015157        1.000000   0.000000e+00
97       0.955847      0.015143        1.000000   0.000000e+00
98       0.955919      0.015079        1.000000   0.000000e+00
99       0.956112      0.015202        1.000000   0.000000e+00

[100 rows x 4 columns]

In [308]:
2 * 0.9556 - 1

0.9112

In [109]:
%%time

models = []
for i in range(30):
    m = xgb.XGBClassifier(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.07,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=40,
    nthread=-1,
    objective='multi:softmax',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=i,
    silent=True,
    subsample=0.8)
    m.fit(all_fold_distances, all_fold_types)
    models.append(m)

CPU times: user 5min 49s, sys: 80 ms, total: 5min 49s
Wall time: 43.8 s


In [110]:
train_dist, test_dist = load_fold('full_distances')
results = [m.predict_proba(test_dist) for m in models]

In [119]:
result = np.mean(results, axis=0)

In [124]:
save_result(test.id.values, result, '../input/submission1.csv') # 911996.56 

In [131]:
clf = LogisticRegression(C=1.0)
search = model_selection.GridSearchCV(
    clf,
    {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100, 1000],
    },
    scoring='roc_auc',
    n_jobs=-1,
    cv=3,
    verbose=3,
)
search.fit(all_fold_distances, all_fold_types == 0)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] penalty=l1, C=0.1 ...............................................
[CV] penalty=l1, C=0.1 ...............................................
[CV] penalty=l1, C=0.1 ...............................................
[CV] penalty=l2, C=0.1 ...............................................
[CV] penalty=l2, C=0.1 ...............................................
[CV] penalty=l2, C=0.1 ...............................................
[CV] penalty=l1, C=1 .................................................
[CV] penalty=l1, C=1 .................................................
[CV] ................ penalty=l2, C=0.1, score=0.803463, total=   3.8s
[CV] penalty=l1, C=1 .................................................
[CV] ................ penalty=l2, C=0.1, score=0.863850, total=   3.9s
[CV] penalty=l2, C=1 .................................................
[CV] ................ penalty=l2, C=0.1, score=0.789252, total=   4.3s
[CV] penalty=l2,

[Parallel(n_jobs=-1)]: Done  26 out of  30 | elapsed:  4.9min remaining:   45.1s


[CV] ............... penalty=l1, C=1000, score=0.845021, total= 2.9min
[CV] ................ penalty=l1, C=100, score=0.840128, total= 4.2min
[CV] ................ penalty=l1, C=100, score=0.800606, total= 3.9min
[CV] ............... penalty=l1, C=1000, score=0.799339, total= 1.7min


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.4min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=3)

In [201]:
xgb_matrix = xgb.DMatrix(all_fold_distances, (all_fold_types == 0).astype(int))
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 2
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.75
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 100

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
    early_stopping_rounds=5,
    feval=xgb_binary_evalerror,
    seed=4,
)

[0]	train-auc:0.984817+0.00216296	test-auc:0.925141+0.0253636
[10]	train-auc:0.997103+0.000392818	test-auc:0.950795+0.0168452
[20]	train-auc:0.998629+0.000298874	test-auc:0.953646+0.0138607
[30]	train-auc:0.999349+0.000180797	test-auc:0.954797+0.0129091
[40]	train-auc:0.999733+0.000102338	test-auc:0.95717+0.0127904
[50]	train-auc:0.9999+4.77733e-05	test-auc:0.958049+0.0126017
[60]	train-auc:0.999976+2.22351e-05	test-auc:0.958663+0.0129981


test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.925141      0.025364        0.984817       0.002163
1        0.939231      0.022187        0.991647       0.000978
2        0.941970      0.019679        0.993388       0.000875
3        0.942165      0.019879        0.994281       0.000966
4        0.945043      0.017665        0.994875       0.000983
5        0.945823      0.017173        0.995414       0.000864
6        0.947622      0.016319        0.995931       0.000558
7        0.948879      0.016021        0.996426       0.000502
8        0.949705      0.016284        0.996616       0.000479
9        0.950722      0.016652        0.996880       0.000503
10       0.950795      0.016845        0.997103       0.000393
11       0.951791      0.016132        0.997318       0.000460
12       0.951344      0.016076        0.997535       0.000371
13       0.951173      0.016041        0.997731       0.000409
14       0.951102      0.015318        0.997920       0.000401
15       0.951716      0.014316        0.998029       0.000404
16       0.952324      0.014381        0.998125       0.000340
17       0.953380      0.013467        0.998273       0.000310
18       0.953134      0.013420        0.998390       0.000305
19       0.953309      0.013285        0.998529       0.000308
20       0.953646      0.013861        0.998629       0.000299
21       0.953502      0.013585        0.998706       0.000305
22       0.953811      0.013240        0.998792       0.000279
23       0.954350      0.013643        0.998873       0.000245
24       0.954714      0.013457        0.998959       0.000226
25       0.954464      0.013366        0.999048       0.000233
26       0.954715      0.013324        0.999127       0.000221
27       0.954667      0.013112        0.999174       0.000208
28       0.954520      0.012848        0.999244       0.000190
29       0.954630      0.012800        0.999309       0.000185
30       0.954797      0.012909        0.999349       0.000181
31       0.955101      0.012942        0.999405       0.000166
32       0.955140      0.012858        0.999464       0.000143
33       0.955617      0.012863        0.999503       0.000135
34       0.955757      0.012969        0.999555       0.000135
35       0.956253      0.012569        0.999583       0.000133
36       0.956361      0.012659        0.999623       0.000124
37       0.956414      0.012865        0.999664       0.000108
38       0.956868      0.012435        0.999685       0.000107
39       0.956984      0.012698        0.999712       0.000106
40       0.957170      0.012790        0.999733       0.000102
41       0.957199      0.012386        0.999756       0.000097
42       0.957147      0.012140        0.999778       0.000088
43       0.957653      0.012158        0.999794       0.000082
44       0.957681      0.012579        0.999810       0.000078
45       0.957830      0.012279        0.999832       0.000067
46       0.957746      0.012214        0.999843       0.000067
47       0.958022      0.012357        0.999856       0.000066
48       0.958024      0.012337        0.999873       0.000057
49       0.958023      0.012482        0.999886       0.000052
50       0.958049      0.012602        0.999900       0.000048
51       0.958215      0.012395        0.999910       0.000049
52       0.958469      0.012508        0.999920       0.000043
53       0.958553      0.012542        0.999930       0.000041
54       0.958692      0.012552        0.999939       0.000033
55       0.958608      0.012350        0.999945       0.000030
56       0.958886      0.012191        0.999953       0.000025

In [215]:
xgb_matrix = xgb.DMatrix(all_fold_distances, (all_fold_types == 1).astype(int))
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 2
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.7
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 100
xgb_param['objective'] = 'binary:logistic'

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
    early_stopping_rounds=5,
    feval=xgb_binary_evalerror,
    seed=4,
)

XGBoostError: b'[11:30:30] src/objective/regression_obj.cc:90: Check failed: (preds.size()) == (info.labels.size()) labels are not correctly providedpreds.size=2160, label.size=1080'

In [196]:
xgb_matrix = xgb.DMatrix(all_fold_distances, (all_fold_types == 2).astype(int))
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 2
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.7
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 100

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
    early_stopping_rounds=5,
    feval=xgb_binary_evalerror,
    seed=4,
)

[0]	train-auc:0.975402+0.0076165	test-auc:0.890552+0.0490033
[10]	train-auc:0.995148+0.00145296	test-auc:0.934735+0.030054
[20]	train-auc:0.997374+0.00116327	test-auc:0.937384+0.0255005
[30]	train-auc:0.998622+0.000930372	test-auc:0.9389+0.0245465
[40]	train-auc:0.999442+0.000311314	test-auc:0.939453+0.0232708
[50]	train-auc:0.999792+9.92613e-05	test-auc:0.940146+0.0234603


test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.890552      0.049003        0.975402       0.007616
1        0.908780      0.038436        0.986549       0.003582
2        0.916876      0.034019        0.990232       0.002255
3        0.917350      0.030631        0.990895       0.002651
4        0.926397      0.028334        0.991973       0.002037
5        0.927898      0.029382        0.993125       0.002230
6        0.930989      0.030251        0.994139       0.001958
7        0.931390      0.030131        0.994494       0.001855
8        0.931875      0.032804        0.994721       0.001709
9        0.933755      0.031081        0.994969       0.001580
10       0.934735      0.030054        0.995148       0.001453
11       0.935094      0.029645        0.995381       0.001390
12       0.935525      0.028360        0.995700       0.001464
13       0.936079      0.027964        0.995934       0.001489
14       0.936034      0.027334        0.996281       0.001448
15       0.936738      0.027743        0.996414       0.001438
16       0.937220      0.027744        0.996521       0.001407
17       0.936930      0.027815        0.996987       0.001302
18       0.937440      0.026342        0.997071       0.001267
19       0.937197      0.026573        0.997125       0.001246
20       0.937384      0.025500        0.997374       0.001163
21       0.938215      0.025338        0.997466       0.001162
22       0.937833      0.025360        0.997516       0.001179
23       0.938326      0.024920        0.997800       0.001097
24       0.938365      0.025729        0.998037       0.000924
25       0.938035      0.025191        0.998151       0.000905
26       0.938595      0.024509        0.998246       0.000927
27       0.938595      0.024540        0.998341       0.000950
28       0.938830      0.024254        0.998419       0.000944
29       0.938942      0.024533        0.998461       0.000944
30       0.938900      0.024546        0.998622       0.000930
31       0.939095      0.024500        0.998858       0.000553
32       0.939100      0.024106        0.998952       0.000485
33       0.938896      0.024213        0.999018       0.000483
34       0.939158      0.023755        0.999077       0.000487
35       0.939442      0.024289        0.999121       0.000498
36       0.939222      0.024123        0.999166       0.000472
37       0.939026      0.023945        0.999234       0.000422
38       0.939412      0.023710        0.999304       0.000409
39       0.939526      0.023541        0.999393       0.000338
40       0.939453      0.023271        0.999442       0.000311
41       0.939290      0.023586        0.999488       0.000265
42       0.939213      0.023681        0.999542       0.000230
43       0.939414      0.023399        0.999577       0.000215
44       0.939951      0.023116        0.999618       0.000187
45       0.940154      0.022983        0.999647       0.000173
46       0.940381      0.022787        0.999690       0.000152

In [197]:
xgb_matrix = xgb.DMatrix(all_fold_distances, (all_fold_types == 4).astype(int))
xgb_param = gbm.get_xgb_params()
xgb_param['num_class'] = 2
xgb_param['seed'] = 1
xgb_param['gamma'] = 0.75
xgb_param['learning_rate'] = 0.05
xgb_param['n_estimators'] = 100

xgb.cv(
    xgb_param,
    xgb_matrix,
    num_boost_round=xgb_param['n_estimators'],
    nfold=10,
    verbose_eval=10,
    early_stopping_rounds=5,
    feval=xgb_binary_evalerror,
    seed=4,
)

[0]	train-auc:0.98706+0.0065002	test-auc:0.935573+0.0425532
[10]	train-auc:0.995164+0.00267166	test-auc:0.954126+0.0306867
[20]	train-auc:0.998093+0.00153089	test-auc:0.956528+0.031794
[30]	train-auc:0.999496+0.00028862	test-auc:0.960624+0.0294616
[40]	train-auc:0.999816+0.00011846	test-auc:0.962522+0.0290579


test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.935573      0.042553        0.987060       0.006500
1        0.944611      0.039858        0.992673       0.003167
2        0.944156      0.041736        0.992746       0.003444
3        0.945204      0.040192        0.993074       0.003108
4        0.949371      0.034056        0.993166       0.003103
5        0.948867      0.033675        0.993684       0.001988
6        0.949586      0.033203        0.993625       0.002021
7        0.952350      0.030409        0.993599       0.001973
8        0.951881      0.029950        0.993661       0.001958
9        0.954812      0.030406        0.994552       0.002709
10       0.954126      0.030687        0.995164       0.002672
11       0.954674      0.031514        0.995280       0.002643
12       0.955215      0.031211        0.995419       0.002587
13       0.955186      0.031032        0.995916       0.002612
14       0.954050      0.031433        0.995983       0.002588
15       0.954309      0.030851        0.996079       0.002608
16       0.954377      0.031346        0.996536       0.002605
17       0.955439      0.032206        0.996586       0.002572
18       0.955837      0.032611        0.997421       0.001962
19       0.956579      0.031911        0.997656       0.002014
20       0.956528      0.031794        0.998093       0.001531
21       0.956830      0.032190        0.998656       0.000586
22       0.957136      0.032420        0.998711       0.000568
23       0.956998      0.032228        0.998831       0.000497
24       0.957391      0.032181        0.998987       0.000429
25       0.957583      0.032700        0.999119       0.000353
26       0.960653      0.029110        0.999230       0.000359
27       0.960113      0.029609        0.999289       0.000342
28       0.960168      0.029412        0.999371       0.000311
29       0.960457      0.029550        0.999432       0.000301
30       0.960624      0.029462        0.999496       0.000289
31       0.960820      0.029877        0.999512       0.000292
32       0.960482      0.030182        0.999557       0.000249
33       0.960833      0.029827        0.999580       0.000271
34       0.961426      0.029482        0.999634       0.000215
35       0.962061      0.028965        0.999674       0.000178
36       0.962358      0.028884        0.999697       0.000191
37       0.962701      0.028549        0.999724       0.000173
38       0.963065      0.028362        0.999766       0.000156

In [216]:
gbm_args = dict(
    base_score=0.5,
    colsample_bylevel=0.9,
    colsample_bytree=0.9,
    gamma=0.65,
    learning_rate=0.03,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    nthread=-1,
    objective='binary:logistic',
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    silent=True,
    subsample=0.8,
)


models_args = [
    # 0
    dict(gbm_args,
#          num_class=2,
         gamma=0.75,
         learning_rate=0.05,
         n_estimators=60
        ),
    # 1
    dict(gbm_args,
#          num_class=2,
         gamma=0.7,
         learning_rate=0.05,
         n_estimators=50
        ),
    # 2
    dict(gbm_args,
#          num_class=2,
         gamma=0.7,
         learning_rate=0.05,
         n_estimators=50
        ),
    # 4
    dict(gbm_args,
#          num_class=2,
         gamma=0.75,
         learning_rate=0.05,
         n_estimators=40
        ),
]

n_seeds = 10
models = [
    [xgb.XGBClassifier(seed=seed + 123, **params) for seed in range(n_seeds)]
    for params in models_args
]

In [217]:
%%time
for idx, ms in zip([0,1,2,4], models):
    y = (all_fold_types == idx).astype(int)
    for m in ms:
        m.fit(all_fold_distances, y)

CPU times: user 2min 8s, sys: 36 ms, total: 2min 8s
Wall time: 16 s


In [225]:
%%time

train_dist, test_dist = load_fold('full_distances')
preds = [
    [m.predict_proba(test_dist)[:, 1] for m in ms]
    for ms in models
]

CPU times: user 1.17 s, sys: 12 ms, total: 1.18 s
Wall time: 160 ms


In [233]:
ranks = [
    [sp.stats.rankdata(p) for p in ps]
    for ps in preds
]

In [255]:
rank_probas = [np.mean(rs, axis=0) for rs in ranks]
for p in rank_probas:
    p /= np.max(p) * 4
other_probas = 1 - np.sum(rank_probas, axis=0)

In [259]:
result = np.vstack(rank_probas[:-1] + [other_probas] + rank_probas[-1:]).T
result.shape

(1211, 5)

In [260]:
result

array([[ 0.06015622,  0.17779477,  0.20872963,  0.40012288,  0.1531965 ],
       [ 0.24327048,  0.0220874 ,  0.03934745,  0.60760779,  0.08768688],
       [ 0.10628648,  0.23191252,  0.10801423,  0.48459127,  0.06919551],
       ..., 
       [ 0.0123357 ,  0.1542087 ,  0.09310644,  0.4957386 ,  0.24461056],
       [ 0.11500021,  0.18521581,  0.15759656,  0.45623508,  0.08595234],
       [ 0.24401587,  0.0220874 ,  0.01566248,  0.65786628,  0.06036797]])

In [261]:
save_result(test.id.values, result, '../input/submission2.csv') # 908811.06

In [263]:
mean_probas = [np.mean(p, axis=0) / 4 for p in preds]
other_probas = 1 - np.sum(mean_probas, axis=0)
result = np.vstack(mean_probas[:-1] + [other_probas] + mean_probas[-1:]).T
result.shape

(1211, 5)

In [264]:
save_result(test.id.values, result, '../input/submission3.csv') # 908878.57